**File: buildComplexesFromPairwiseScoresRecursive.py**
---
*   Author: Jose Lugo-Martinez
*   Date: July 03, 2018
*   Advisors: Robert Murphy and Ziv Bar-Joseph

> Example sample call:

> python buildComplexesFromPairwiseScoresRecursive.py pairsInfoFile pairsPredictionsFile cutoff target% outputPATH ./results/






#install packages and load libraries--gw added

Install non-native packages



In [1]:
%%capture
!pip install -U scikit-learn
!pip install ipynb

Import common packages

In [2]:
import copy, cProfile, datetime, glob
import itertools
import matplotlib.pyplot as plt
import math
import networkx as nx
import numpy as np
import os
import pandas as pd
import pickle, pstats, re
import seaborn as sns
import shutil
from statistics import mean
import subprocess, sys
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, average_precision_score, f1_score
from sklearn.metrics import plot_precision_recall_curve, precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, train_test_split
import time

Enable Google Colab, mount drives, and load proprietary modules...

In [3]:
%%capture
from google.colab import drive, files
drive.mount('/content/drive', force_remount=False)
sys.path.insert(0, '/content/drive/My Drive/Primary Research/JLMwSCBC_notebook/code/code_toPublish')                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

#define file paths and key variables--gw added

In [ ]:
from getAllCliques_gwMods import getAllCliques_gwMods
#directories
workDir = '/content/drive/My Drive/Primary Research/JLMwSCBC_notebook/'
complexesPath = workDir + 'complexPredictions/'

CORUMcomplexes_dataPath = \
  workDir + 'proteinComplexes/coreComplexes_03-09-2018_Corum-3.0 .txt'
drew2017_complexesDir = workDir + 'proteinComplexes/Drew 2017 Data Sources/'
humap1_complexesDir = workDir + 'proteinComplexes/huMAP_1.0/'
humap2_complexesDir = workDir + 'proteinComplexes/huMAP_2.0/'

LMprotein_complexesDir = workDir + 'proteinComplexes/Lugo-Martinez/'
GWprotein_complexesDir = workDir + 'proteinComplexes/Drew_LM_Wilkins/'
modelsPath_path = workDir + 'models/'
modelPerformances_path = workDir + 'modelPerformance/modelsPerformance_'

expName = \
  'integratedPairs_KD+LM+FGK+SCBC_cellSpecific_nci60Feats_noSCBCnulls-5CV'
cellLines = ['cellLines_mean', 'H322', 'MCF7', 'U251']
#ppiThresholds = ['top5%', 'top10%', 'top15%', 'top20%', 'top25%']
dirSuffix = '_predictedComplexes'

In [ ]:
expName = \
  'integratedPairs_KD+LM+FGK+SCBC_cellSpecific_nci60Feats_noSCBCnulls-5CV'
modelPerformance_dir = \
  workDir + \
  'modelPerformance/modelsPerformance_{0}/complexPredictions/'.format(expName)
pairsInfo_path = \
  modelPerformance_dir + 'expandedPairset_cellSpecific_KD+LM+FGK_pairsInfo.tsv'
#cellTypes = ['cellLines_mean', 'H322', 'MCF7', 'U251']
cellTypes = ['MCF7']
for cellType in cellTypes:

    pairsPredictions_path = modelPerformance_dir + \
      'expandedPairset_probsPos_{0}'.format(cellType) + \
      'Specific_KD+LM+FGK_pairsPredictions'
    outputPath = modelPerformance_dir + '{0}/'.format(cellType)
    getAllCliques_gwMods(pairsInfo_path, 
                         pairsPredictions_path, 
                         0.015, 
                         outputPath)

# of nodes 1679
# of edges 383430


In [ ]:
#directories
workDir = '/content/drive/My Drive/Colab Notebooks/Primary Research/JLMwSCBC_notebook/'
modelPerformances_path = workDir + 'modelPerformance/modelsPerformance_'
complexesPath = workDir + 'proteinComplexes/'

In [ ]:
#define key variables, e.g. experiment name, filename components, etc.,
expName = 'integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV'
cellLines = ['h322m', 'mcf7', 'u251']
pairsInfo_fileSuffix = '-specific_pairsInfo_threshold=0.328.tsv' 
pairsScores_fileSuffix = '-specific_pairPredictions_threshold=0.328'
#outputPATH = modelPerformances_path + expName + '/' + 'buildProtein_complexes/' #top 0.02%
#os.makedirs(outputPATH)
MAX_K = 45
cutoff = 0.328
targetPercentile = 50.0
pairsScore_threshold = 0

#file/data prep for LM code exec

In [ ]:
meanDiff_weightedDF = pd.read_csv(modelPerformances_path + expName + '/' + 'cellType_specificProps_varied.tsv', sep='\t')

In [ ]:
meanDiff_weightedDF.shape

(1038169, 36)

In [ ]:
meanDiff_weightedDF.dtypes

h322m @0.400    float64
h322m @0.466    float64
h322m @0.531    float64
h322m @0.597    float64
h322m @0.662    float64
h322m @0.728    float64
h322m @0.793    float64
h322m @0.859    float64
h322m @0.924    float64
h322m @0.990    float64
mcf7 @0.400     float64
mcf7 @0.466     float64
mcf7 @0.531     float64
mcf7 @0.597     float64
mcf7 @0.662     float64
mcf7 @0.728     float64
mcf7 @0.793     float64
mcf7 @0.859     float64
mcf7 @0.924     float64
mcf7 @0.990     float64
u251 @0.400     float64
u251 @0.466     float64
u251 @0.531     float64
u251 @0.597     float64
u251 @0.662     float64
u251 @0.728     float64
u251 @0.793     float64
u251 @0.859     float64
u251 @0.924     float64
u251 @0.990     float64
h322m_probs     float64
mcf7_probs      float64
u251_probs      float64
labels          float64
IDi               int64
IDii              int64
dtype: object

In [ ]:
meanDiff_weightedDF.head()

,h322m @0.400,h322m @0.466,h322m @0.531,h322m @0.597,h322m @0.662,h322m @0.728,h322m @0.793,h322m @0.859,h322m @0.924,h322m @0.990,mcf7 @0.400,mcf7 @0.466,mcf7 @0.531,mcf7 @0.597,mcf7 @0.662,mcf7 @0.728,mcf7 @0.793,mcf7 @0.859,mcf7 @0.924,mcf7 @0.990,u251 @0.400,u251 @0.466,u251 @0.531,u251 @0.597,u251 @0.662,u251 @0.728,u251 @0.793,u251 @0.859,u251 @0.924,u251 @0.990,h322m_probs,mcf7_probs,u251_probs,labels,IDi,IDii
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.080337,0.083751,0.080989,1.0,27316,24144
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058749,0.148428,0.105925,1.0,7846,1938
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024688,0.062504,0.020001,1.0,56949,4670
3,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.781380,0.874401,0.714871,1.0,6205,6136
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.056897,0.055342,0.027523,1.0,58517,6633


In [ ]:
meanDiff_weightedDF[['IDi', 'IDii', 'labels', 'h322m_probs', 'mcf7_probs', 'u251_probs']].head()

,IDi,IDii,labels,h322m_probs,mcf7_probs,u251_probs
0,27316,24144,1.0,0.080337,0.083751,0.080989
1,7846,1938,1.0,0.058749,0.148428,0.105925
2,56949,4670,1.0,0.024688,0.062504,0.020001
3,6205,6136,1.0,0.781380,0.874401,0.714871
4,58517,6633,1.0,0.056897,0.055342,0.027523


In [ ]:
pairsAbove_thresholdJLM = pd.read_csv(workDir + 'proteinComplexes/Lugo-Martinez/integrated_MM+TSCS+Exp+Abun+Loc_features_all_pairs_model_either_predictions_rf_candidates.txt', sep=',|\t', header=None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
pairsAbove_thresholdJLM.shape

(20615986, 3)

In [ ]:
pairsAbove_thresholdJLM.head()

,0,1,2
0,114781,114787,0.47941
1,114787,114789,0.48046
2,114787,28991,0.47972
3,114787,28992,0.49071
4,114787,55293,0.48293


In [ ]:
pairsAbove_thresholdJLM.dtypes

0     object
1     object
2    float64
dtype: object

In [ ]:
meanDiff_weightedDF_conv = meanDiff_weightedDF.astype({'labels': 'int64', 'IDi': 'str', 'IDii': 'str'})

In [ ]:
meanDiff_weightedDF_conv = meanDiff_weightedDF_conv[['IDi', 'IDii', 'labels', 'h322m_probs', 'mcf7_probs', 'u251_probs']]

In [ ]:
meanDiff_weightedDF_conv.shape

(1038169, 6)

In [ ]:
meanDiff_weightedDF_conv.head()

,IDi,IDii,labels,h322m_probs,mcf7_probs,u251_probs
0,27316,24144,1,0.080337,0.083751,0.080989
1,7846,1938,1,0.058749,0.148428,0.105925
2,56949,4670,1,0.024688,0.062504,0.020001
3,6205,6136,1,0.781380,0.874401,0.714871
4,58517,6633,1,0.056897,0.055342,0.027523


In [ ]:
pairsAll = pairsAbove_thresholdJLM.apply(lambda x: set(x[[0, 1]]), axis=1)

In [ ]:
pairsPrtn = meanDiff_weightedDF_conv.loc[:, ['IDi', 'IDii']].apply(lambda x: set(x[[0, 1]]), axis=1)

In [ ]:
pairsPrtn.head()

0    {24144, 27316}
1      {1938, 7846}
2     {56949, 4670}
3      {6205, 6136}
4     {6633, 58517}
dtype: object

In [ ]:
meanDiff_weightedDF_conv['proteinPairs'] = pairsPrtn

In [ ]:
len(set(pd.concat([pairsAbove_thresholdJLM.sort_values(by=2, ascending=False).iloc[:round( len(pairsAbove_thresholdJLM) * 0.0002), 0], pairsAbove_thresholdJLM.sort_values(by=2, ascending=False).iloc[:round( len(pairsAbove_thresholdJLM) * 0.0002), 1]]).to_list()))

896

In [ ]:
pairsAll.loc[0]

{'114781', '114787'}

In [ ]:
meanDiff_weightedDF_conv.dtypes

IDi             object
IDii            object
labels           int64
h322m_probs    float64
mcf7_probs     float64
u251_probs     float64
dtype: object

In [ ]:
meanDiff_weightedDF_conv['LM_probs'] = np.nan

In [ ]:
meanDiff_weightedDF_conv.head()

,proteinPairs,IDi,IDii,labels,h322m_probs,mcf7_probs,u251_probs,LM_probs
0,"{24144, 27316}",27316,24144,1,0.080337,0.083751,0.080989,NaN
1,"{1938, 7846}",7846,1938,1,0.058749,0.148428,0.105925,NaN
2,"{56949, 4670}",56949,4670,1,0.024688,0.062504,0.020001,NaN
3,"{6205, 6136}",6205,6136,1,0.781380,0.874401,0.714871,NaN
4,"{6633, 58517}",58517,6633,1,0.056897,0.055342,0.027523,NaN


In [ ]:
pairsAbove_thresholdJLM['proteinPairs'] = pairsAll

In [ ]:
pairsAbove_thresholdJLM.head()

,proteinPairs,IDi,IDii,2
0,"{114781, 114787}",114781,114787,0.47941
1,"{114787, 114789}",114787,114789,0.48046
2,"{28991, 114787}",114787,28991,0.47972
3,"{28992, 114787}",114787,28992,0.49071
4,"{114787, 55293}",114787,55293,0.48293


In [ ]:
meanDiff_weightedDF_conv.head()

,proteinPairs,IDi,IDii,labels,h322m_probs,mcf7_probs,u251_probs
0,"{24144, 27316}",27316,24144,1,0.080337,0.083751,0.080989
1,"{1938, 7846}",7846,1938,1,0.058749,0.148428,0.105925
2,"{56949, 4670}",56949,4670,1,0.024688,0.062504,0.020001
3,"{6205, 6136}",6205,6136,1,0.781380,0.874401,0.714871
4,"{6633, 58517}",58517,6633,1,0.056897,0.055342,0.027523


In [ ]:
intersectingPairs_1 = meanDiff_weightedDF_conv.merge(pairsAbove_thresholdJLM.loc[:, ['IDi', 'IDii', 2]], left_on=['IDi', 'IDii'], right_on=['IDi', 'IDii'])
intersectingPairs_2 = meanDiff_weightedDF_conv.merge(pairsAbove_thresholdJLM.loc[:, ['IDi', 'IDii', 2]], left_on=['IDii', 'IDi'], right_on=['IDi', 'IDii'])

In [ ]:
520197+517972

1038169

In [ ]:
intersectingPairs_1.head()

,proteinPairs,IDi,IDii,labels,h322m_probs,mcf7_probs,u251_probs,LM_probs,2
0,"{6633, 58517}",58517,6633,1,0.056897,0.055342,0.027523,NaN,0.65162
1,"{10236, 9785}",10236,9785,1,0.122475,0.123116,0.116867,NaN,0.61300
2,"{26589, 64949}",26589,64949,1,0.982971,0.980408,0.977173,NaN,0.63485
3,"{55596, 23450}",23450,55596,1,0.232806,0.244692,0.208102,NaN,0.54609
4,"{84991, 3329}",3329,84991,1,0.007188,0.003125,0.005626,NaN,0.51448


In [ ]:
intersectingPairs_2.head()

,proteinPairs,IDi_x,IDii_x,labels,h322m_probs,mcf7_probs,u251_probs,LM_probs,IDi_y,IDii_y,2
0,"{24144, 27316}",27316,24144,1,0.080337,0.083751,0.080989,0.50368,24144,27316,0.50368
1,"{1938, 7846}",7846,1938,1,0.058749,0.148428,0.105925,NaN,1938,7846,0.55580
2,"{56949, 4670}",56949,4670,1,0.024688,0.062504,0.020001,NaN,4670,56949,0.47950
3,"{6205, 6136}",6205,6136,1,0.781380,0.874401,0.714871,NaN,6136,6205,0.86103
4,"{24148, 26986}",26986,24148,1,0.023439,0.016563,0.018125,NaN,24148,26986,0.47529


In [ ]:
intersectingPairs_1 = intersectingPairs_1[['IDi', 'IDii', 'labels', 'h322m_probs', 'mcf7_probs', 'u251_probs', 2]]
intersectingPairs_2 = intersectingPairs_2[['IDi_x', 'IDii_x', 'labels', 'h322m_probs', 'mcf7_probs', 'u251_probs', 2]]
intersectingPairs_2.rename(columns={'IDi_x': 'IDi', 'IDii_x': 'IDii'}, inplace=True)

In [ ]:
intersectingPairs_1.head()

,IDi,IDii,labels,h322m_probs,mcf7_probs,u251_probs,2
0,58517,6633,1,0.056897,0.055342,0.027523,0.65162
1,10236,9785,1,0.122475,0.123116,0.116867,0.61300
2,26589,64949,1,0.982971,0.980408,0.977173,0.63485
3,23450,55596,1,0.232806,0.244692,0.208102,0.54609
4,3329,84991,1,0.007188,0.003125,0.005626,0.51448


In [ ]:
intersectingPairs_2.head()

,IDi,IDii,labels,h322m_probs,mcf7_probs,u251_probs,2
0,27316,24144,1,0.080337,0.083751,0.080989,0.50368
1,7846,1938,1,0.058749,0.148428,0.105925,0.55580
2,56949,4670,1,0.024688,0.062504,0.020001,0.47950
3,6205,6136,1,0.781380,0.874401,0.714871,0.86103
4,26986,24148,1,0.023439,0.016563,0.018125,0.47529


In [ ]:
intersectingPairs = pd.concat([intersectingPairs_1, intersectingPairs_2])

In [ ]:
intersectingPairs.head()

,IDi,IDii,labels,h322m_probs,mcf7_probs,u251_probs,2
0,58517,6633,1,0.056897,0.055342,0.027523,0.65162
1,10236,9785,1,0.122475,0.123116,0.116867,0.61300
2,26589,64949,1,0.982971,0.980408,0.977173,0.63485
3,23450,55596,1,0.232806,0.244692,0.208102,0.54609
4,3329,84991,1,0.007188,0.003125,0.005626,0.51448


In [ ]:
#pickle.dump(intersectingPairs, open(complexesPath + 'LM_pairsPredictions_cv.pkl', 'wb'))
intersectingPairs = pickle.load(open(complexesPath + 'LM_pairsPredictions_cv.pkl', 'rb'))

In [ ]:
intersectingPairs.shape

(1038169, 7)

In [ ]:
intersectingPairs.loc[intersectingPairs[2] > 0.87].shape

(2910, 7)

In [ ]:
len(set(pd.concat([intersectingPairs.loc[intersectingPairs[2] > 0.87, 'IDi'], intersectingPairs.loc[intersectingPairs[2] > 0.87, 'IDii']]).to_list()))

566

In [ ]:
intersectingPairs.loc[:, 2].isna().sum()

0

In [ ]:
for cell in cellLines:
  print('shape of pairsInfo file: ' + str(meanDiff_weightedDF.loc[meanDiff_weightedDF[cell + '_probs'] > pairsScore_threshold, ['labels', 'IDi', 'IDii']].shape))
  meanDiff_weightedDF.loc[meanDiff_weightedDF[cell + '_probs'] > pairsScore_threshold, ['labels', 'IDi', 'IDii']].to_csv(workDir + 'modelPerformance/modelsPerformance_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV/{0}-specific_pairsInfo_threshold={1}.tsv'.format(cell, pairsScore_threshold), sep='\t', index=False, header=False)
  print('shape of pairsInfo scores file: ' + str(meanDiff_weightedDF.loc[meanDiff_weightedDF[cell + '_probs'] >pairsScore_threshold, [cell + '_probs']].shape))
  meanDiff_weightedDF.loc[meanDiff_weightedDF[cell + '_probs'] > pairsScore_threshold, [cell + '_probs']].to_csv(workDir + 'modelPerformance/modelsPerformance_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV/{0}-specific_pairPredictions_threshold={1}.txt'.format(cell, pairsScore_threshold), sep='\t', index=False, header=False)

shape of pairsInfo file: (1011118, 3)
shape of pairsInfo scores file: (1011118, 1)
shape of pairsInfo file: (1010440, 3)
shape of pairsInfo scores file: (1010440, 1)
shape of pairsInfo file: (1010268, 3)
shape of pairsInfo scores file: (1010268, 1)


In [ ]:
def get_thresholds_by_fold_wIDs(modelPerformances_dirs, dataPartition_dirs, cellType_names):

  thresholds = np.arange(0, 1, 0.001)
  print('loading pair info...')
  labelsIDs_byFold = {fold: pd.read_csv(dataPartitions_dirs[fold] + 'testLabels.tsv', delimiter='\t', usecols=[1, 2]) for fold in np.arange(5)}
  for cellType in cellType_names:
    print(cellType)

    pairsPredictions = pd.DataFrame(np.empty((0, 4)), columns=['IDi', 'IDii', 'labels', 'probs'])

    print('loading labels...')
    labels_byFold = {fold: np.loadtxt(modelPerformances_dirs[fold] + 'testLabels_adj_{0}.csv'.format(cellType), delimiter=',') for fold in np.arange(5)}

    print('loading allPreds_posProbs...')
    probsPos_combinedWeighted_byFolds = {fold: np.loadtxt(modelPerformances_dirs[fold] + 'probsPos_weighted_{0}.csv'.format(cellType), delimiter=',') for fold in np.arange(5)}

    for fold in np.arange(5):
      appendingPredictions = pd.DataFrame(labelsIDs_byFold[fold], columns=['IDi', 'IDii'])
      appendingPredictions['labels'] = labels_byFold[fold]
      appendingPredictions['probs'] = probsPos_combinedWeighted_byFolds[fold]
      print('printing appendingPredictions shape...')
      print(appendingPredictions.shape)
      print('printing the head of appendingPredictions...')
      print(appendingPredictions.head())

      fscore_weighted = [f1_score(labels_byFold[fold], to_labels(probsPos_combinedWeighted_byFolds[fold], t)) for t in thresholds]
      ixWeighted = np.nanargmax(fscore_weighted)  ##should be single value
      optimalThreshold = thresholds[ixWeighted]  ##should be single value

      appendingPredictions = appendingPredictions.loc[appendingPredictions.probs >= optimalThreshold, :]
      print('printing appendingPredictions shape after thresholding...')
      print(appendingPredictions.shape)

      pairsPredictions = pairsPredictions.append(appendingPredictions)

    print('printing pairsPredictions shape for cell line {0}: {1}'.format(cellType, pairsPredictions.shape))
    print('printing the head of pairsPredictions for cell line {0}: {1}'.format(cellType, pairsPredictions.head()))
    os.makedirs(workDir + 'modelPerformance/modelsPerformance_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV/pairsPredictions_NEW/', exist_ok=True)
    pairsPredictions[['labels', 'IDi', 'IDii']].to_csv(workDir + 'modelPerformance/modelsPerformance_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV/pairsPredictions_NEW/{0}-specific_pairsInfo.tsv'.format(cell), sep='\t', index=False, header=False)
    pairsPredictions.probs.to_csv(workDir + 'modelPerformance/modelsPerformance_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV/pairsPredictions_NEW/{0}-specific_pairPredictions.txt'.format(cell), sep='\t', index=False, header=False)


In [ ]:
get_thresholds_by_fold_wIDs(modelPerformances_dirs, dataPartitions_dirs, cellType_names)

#exec LM code

#find threshold for 0.02% high-scoring pairs

In [ ]:
intersectingPairs_sorted = intersectingPairs.sort_values(by=[2], ascending=False)

In [ ]:
cutPoint = round(len(intersectingPairs_sorted)*0.02)
print(cutPoint)
intersectingPairs_sorted.iloc[cutPoint, :]

20763


IDi                   7529
IDii                  3913
labels                   0
h322m_probs              0
mcf7_probs               0
u251_probs     0.000312567
2                  0.66552
Name: 502488, dtype: object

In [ ]:
for cell, cellNum in zip(['h322m', 'mcf7', 'u251'], [3, 4, 5]):
  print(cell)
  print('# pairs...')
  print(len(intersectingPairs.loc[intersectingPairs[cell+'_probs']>=0.328, :].sort_values(by=[cell+'_probs'], ascending=False).iloc[:( round( len(intersectingPairs.loc[intersectingPairs[cell+'_probs']>=0.328, :])*0.25))]))
  print('threshold is...')
  print(intersectingPairs.loc[intersectingPairs[cell+'_probs']>=0.328, :].sort_values(by=[cell+'_probs'], ascending=False).iloc[round( len(intersectingPairs.loc[intersectingPairs[cell+'_probs']>=0.328, :])*0.25), cellNum])
  print('! proteins...')
  print(
      len(
          set(
              pd.concat(
                  [intersectingPairs.loc[intersectingPairs[cell+'_probs']>=0.328, :].sort_values(by=[cell+'_probs'], ascending=False).iloc[:( round( len(intersectingPairs.loc[intersectingPairs[cell+'_probs']>=0.328, :])*0.25)), 0], 
                   intersectingPairs.loc[intersectingPairs[cell+'_probs']>=0.328, :].sort_values(by=[cell+'_probs'], ascending=False).iloc[:( round( len(intersectingPairs.loc[intersectingPairs[cell+'_probs']>=0.328, :])*0.25)), 1]] 
                  ).to_list()
              )
          )
      )

h322m
# pairs...
3386
threshold is...
0.9319456482364152
! proteins...
485
mcf7
# pairs...
3554
threshold is...
0.9318842721056818
! proteins...
537
u251
# pairs...
3485
threshold is...
0.9328605831399459
! proteins...
501


##find threshold for 0.05% high-scoring pairs

In [ ]:
ribosomeConstituents = set(pickle.load(open(complexesPath + 'ribosomeConstituents.pkl', 'rb')))

In [ ]:
len(ribosomeConstituents)

In [ ]:
pairsInfoFilename = modelPerformances_path + expName + '/' + 'h322m' + pairsInfo_fileSuffix
pairsInfo = pd.read_csv(pairsInfoFilename, sep='\t', header=None)

In [ ]:
pairsInfo.shape

In [ ]:
cellLines

['h322m', 'mcf7', 'u251']

In [ ]:
for cell in cellLines:
  print(cell)
  #pairsPredictionsFilename = modelPerformances_path + expName + '/' + cell + pairsScores_fileSuffix + '.txt'
  pairsInfoFilename = modelPerformances_path + expName + '/' + cell + pairsInfo_fileSuffix
  #print(pairsPredictionsFilename)
  #pairsPredictions = pd.read_csv(pairsPredictionsFilename, header=None)
  pairsInfo = pd.read_csv(pairsInfoFilename, sep='\t', header=None)
  #combined = pd.concat([pairsInfo, pairsPredictions], axis=1)
  combinedProteins = pd.concat([pairsInfo.iloc[:, 1], pairsInfo.iloc[:, 2]], axis=0)
  combinedProteins.drop_duplicates(inplace=True)
  #print(pairsPredictions.shape)
  print(pairsInfo.shape)
  #print(combined.shape)
  combinedProteins = set(combinedProteins.to_list())
  print(len(combinedProteins))
  intersectionFraction = ribosomeConstituents.intersection(combinedProteins)
  print('the intersection of ribsome complex constituent proteins with unique proteins across all pairs meeting f1 score based criteria is : {0}'.format(intersectionFraction))
  break
  
  '''
  for quantile in np.linspace(0.75, 0.95, 5):
    cutoff = combined.iloc[:, 3].quantile(quantile)
    print(cutoff)
    combined.loc[combined.iloc[:, 3] >= quantile].shape
    combined_idx = combined.loc[combined.iloc[:, 3] >= quantile].index.to_numpy()
    #print(combined_idx.shape)
    remainingProteins = combined.loc[combined.iloc[:, 3] >= quantile].iloc[:, [1, 2]]
    consolidatedProteins = pd.concat([remainingProteins.iloc[:, 0], remainingProteins.iloc[:, 1]], axis=0)
    consolidatedProteins.drop_duplicates(inplace=True)
    print(consolidatedProteins.shape)
  '''

In [ ]:
percentileDir = 25
complexes = dict()
membership = dict()

for key in glob.glob(modelPerformances_path + expName + '/top' + str(percentileDir) + '%/' + cell + '_predictedComplexes/' + cell + pairsScores_fileSuffix + '_predicted_complexesSize_*.txt'):
  keyIdx = key.split('/')[-1].split('_')[-1].split('.')[0]
  complexes[keyIdx] = pd.read_csv(key, sep=',|\t', usecols=list(np.arange(int(key.split('/')[-1].split('_')[-1].split('.')[0]))), header=None)
  print('complexes shape...')
  print(complexes[keyIdx].shape)
  membership[keyIdx] = np.empty(len(complexes[keyIdx]), )
  print('initial intersection reporting container shape...')
  print(membership[keyIdx].shape)
  for idx in np.arange(len(complexes[keyIdx])):
    membership[keyIdx][idx] = len(intersectionFraction.intersection(set(complexes[keyIdx].iloc[idx, :].to_list())))


In [ ]:
membership[key].max()

In [ ]:
for key in membership.keys():
  print('key :' + key + ', ' + '# of predicted complexes :' + str(complexes[key].shape[0]))
  print(str((membership[key] > 0).sum()) + ' representing ' + str( int((membership[key] > 0).sum()/complexes[key].shape[0] * 100)) )
  print(membership[key].max())

In [ ]:
cell = 'h322m'
pairsPredictionsFilename = modelPerformances_path + expName + '/' + cell + pairsScores_fileSuffix + '.txt'
print(pairsPredictionsFilename)

In [ ]:
pairsPredictions = pd.read_csv(pairsPredictionsFilename, header=None)
print(pairsPredictions.shape)

In [ ]:
pairsPredictions.head()

In [ ]:
quantile = 0.90

In [ ]:
cutoff = pairsPredictions.quantile(quantile).iloc[0]
print(cutoff)

In [ ]:
pairsPredictions.loc[pairsPredictions[0] >= quantile].shape
pairsPredictions_idx = pairsPredictions.loc[pairsPredictions[0] >= quantile].index.to_numpy()

In [ ]:
pairsPredictions_idx.shape

In [ ]:
for q in np.linspace(0.65, 0.99, 10):
  cutThreshold = pairsPredictions.quantile(q).iloc[0]
  pairsPredictions.loc[pairsPredictions[0] >= q].shape
  pairsPredictions_idx = pairsPredictions.loc[pairsPredictions[0] >= q].index.to_numpy()
  print(q, pairsPredictions_idx.shape)

##reorganized Jose's code

In [ ]:
cellLines = ['h322m']

In [ ]:
for cell in cellLines:
  pairsInfoFilename = modelPerformances_path + expName + '/' + cell + pairsInfo_fileSuffix
  pairsPredictionsFilename = modelPerformances_path + expName + '/' + cell + pairsScores_fileSuffix

  TOP_CUTOFF = cutoff   

  #Open input file
  try:
      infilePairsInfo = open(pairsInfoFilename, "r")
  except IOError as e:
      print("<<ERROR>> Unable to open the file", pairsInfoFilename, "\nThis program will be quiting now."), e
      sys.exit()

  #Open input file
  predictionsFilename = pairsPredictionsFilename + '.txt'
  try:
      infileScores = open(predictionsFilename, "r")
  except IOError as e:
      print("<<ERROR>> Unable to open the file", predictionsFilename, "\nThis program will be quiting now."), e
      sys.exit()

  proteins = {}
  restrictedProteins = {}
  currComplexes = {}
  scores = {}
  comparisonMatrix = {}
  outfilename = pairsPredictionsFilename + '_candidates_complexesSize_2.txt'
  outfile = open(outfilename, 'w')
  for line in infileScores:
      line = line.strip()
      tokens = line.split('\t')
      pScore = float(tokens[0])

      infoLine = infilePairsInfo.readline().strip()
      tokens = infoLine.split('\t')
      classLabel = tokens[0]
      prot1ID = tokens[1]
      prot2ID = tokens[2]
      pair1 = (prot1ID, prot2ID)
      pair2 = (prot2ID, prot1ID)
      comparisonMatrix[pair1] = pScore

      if not (prot1ID in proteins):
          proteins[prot1ID] = prot1ID
      if not (prot2ID in proteins):
          proteins[prot2ID] = prot2ID

      temp = []
      if pScore >= TOP_CUTOFF:
          if not (prot1ID in restrictedProteins):
              restrictedProteins[prot1ID] = prot1ID
          if not (prot2ID in restrictedProteins):
              restrictedProteins[prot2ID] = prot2ID

          if not (prot1ID in temp):
              temp.append(prot1ID)
          if not (prot2ID in temp):
              temp.append(prot2ID)
          temp.sort()
          complexID = ','.join(temp)
          if not (complexID in currComplexes):
              outline = complexID + '\t' + str(pScore) + '\n'
              outfile.writelines(outline)
              currComplexes[complexID] = (temp, pScore)
              scores[complexID] = pScore

  #Close files
  infilePairsInfo.close()
  infileScores.close()
  outfile.close()
  outfilename = pairsPredictionsFilename.split('_features_')[0] + '_candidates_complexesSize_2@' + str(cutoff) + '.gv'
  outfile = open(outfilename, 'w')

  #Prepare headers
  outline = 'graph {' + '\n'
  outline += '\t node [shape = circle];' + '\n'
  outline += '\t edge [style = solid];' + '\n'
  outfile.writelines(outline)
  nodeID = 0
  protID2nodeID = {}
  for protID in restrictedProteins.keys():
      if not (protID in protID2nodeID):
          protID2nodeID[protID] = nodeID
          outline = '\t n' + str(nodeID) + ' [label = p' + str(nodeID) + '];' + '\n'
          outfile.writelines(outline)
          nodeID += 1

  outfilenameGraph = pairsPredictionsFilename.split('_features_')[0] + '_candidates_complexesSize_2@' + str(cutoff) + '.graph'
  outfileGraph = open(outfilenameGraph, 'w')
  outlineGraph = 'p edge ' + str(len(restrictedProteins)) + '\n'
  outfileGraph.writelines(outlineGraph)
  for complexID, attributes in currComplexes.items():
      prot1ID, prot2ID = complexID.split(',')
      edgeScore = copy.copy(attributes[1])
      if prot1ID in protID2nodeID:
          node1ID = protID2nodeID[prot1ID]
      if prot2ID in protID2nodeID:
          node2ID = protID2nodeID[prot2ID]
      outline = '\t n' + str(node1ID) + ' -- ' + 'n' + str(node2ID) + ' [label = ' + str(edgeScore) + '];' + '\n'
      outfile.writelines(outline)
      outlineGraph = 'e ' + str(prot1ID) + ' ' + str(prot2ID) + ' ' + str(edgeScore) + '\n'
      outfileGraph.writelines(outlineGraph)

      #Prepare footers
      outline = '}' + '\n'
  outfile.writelines(outline)

  #Close file
  outfile.close()
  outfileGraph.close()

  ##insert code for "get all cliques"
  infile  = open(outfilenameGraph, "r")
  
  headers = infile.readline().strip()
  
  G = nx.Graph()
  comparisonMatrix = {}
  for line in infile:
      line = line.strip()
      edgeType, prot1ID, prot2ID, pScore = line.split(' ')
      G.add_edge(prot1ID, prot2ID)  
      proteinPair = (prot1ID, prot2ID)
      comparisonMatrix[proteinPair] = float(pScore)
  #Close input file    
  infile.close()
  
  print ("# of nodes", len(G.nodes))
  print ("# of edges", len(G.edges))
  
  cliques = nx.find_cliques(G)

  predictedComplexes = {}
  for cliqueVerticesList in cliques:
      cliqueVerticesList.sort()
      complexID = ','.join(cliqueVerticesList)
      complexSize = len(cliqueVerticesList)
      scoresPerPair = []
      for i in range((len(cliqueVerticesList) - 1)):
          for j in range(i + 1, len(cliqueVerticesList)):
              pairF = (cliqueVerticesList[i], cliqueVerticesList[j])
              pairB = (cliqueVerticesList[j], cliqueVerticesList[i])
              pairScore = 0.0
              if pairF in comparisonMatrix:
                  pairScore = comparisonMatrix[pairF]
              elif pairB in comparisonMatrix:
                  pairScore = comparisonMatrix[pairB]
              scoresPerPair.append(pairScore)
      complexScore = min(scoresPerPair)
      if complexSize in predictedComplexes:
          predictedComplexes[complexSize].append((complexID, complexScore))
      else:
          predictedComplexes[complexSize] = [(complexID, complexScore)]

  print (len(predictedComplexes.keys()))
  #gw change
  outfilenamePrefix = pairsPredictionsFilename
  for complexSize, complexesSet in predictedComplexes.items():
      outfilename = outfilenamePrefix + str(complexSize) + '.txt'
      outfile  = open(outfilename, "w")
      for complexID, complexScore in complexesSet:
          outline = complexID + '\t' + str(complexScore) + '\n'
          outfile.writelines(outline)
      #Close output file    
      outfile.close()
  ##END insert code for "get all cliques"

  '''
  percentile = np.percentile(list(scores.values()), targetPercentile)
  averageWeight = np.mean(list(scores.values()))
  stdWeight = np.std(list(scores.values()))
  weightThreshold = percentile

  print('# of proteins', len(proteins), len(restrictedProteins))
  print('Candidate pairwise interactions', len(comparisonMatrix))
  print('\t High-scoring pairwise interactions', len(currComplexes))
  print('\t Average pairwise score', averageWeight)
  print('\t Standard deviation of pairwise scores', stdWeight)
  outline = '\t ' + str(targetPercentile) + ' percentile score ' + str(weightThreshold)
  print(outline)

  k = 3
  while len(currComplexes) > 0 and k <= MAX_K:
      complexesCount = 0
      candidateComplexes = copy.copy(currComplexes)
      print (k - 1), len(restrictedProteins)
      proteinSet = restrictedProteins.keys()
      restrictedProteins = {}
      scores = {}
      currComplexes = {}
      alreadyPrinted = {}
      outfilenamePredicted = pairsPredictionsFilename + '_predicted_complexesSize_' + str(k-1) + '.txt'
      outfilePredicted = open(outfilenamePredicted, "w")
      for candidateComplex, weight in candidateComplexes.values():
          augmented = False
          for prot2ID in proteinSet:
              prune = False
              temp = []
              for i in range(len(candidateComplex)):
                  prot1ID = candidateComplex[i]
                  if prot2ID in candidateComplex:
                      break
                  pair1 = (prot1ID, prot2ID)
                  pair2 = (prot2ID, prot1ID)
                  if pair1 in comparisonMatrix:
                      pScore = comparisonMatrix[pair1]
                  elif pair2 in comparisonMatrix:
                      pScore = comparisonMatrix[pair2]
                  else:
                      break
                  if pScore >= TOP_CUTOFF: #PAIRWISE_INTERACTIONS_CUTOFF:
                      if not (prot1ID in temp):
                          temp.append(prot1ID)
                      if not (prot2ID in temp):
                          temp.append(prot2ID)
                  else:
                      prune = True
                      break
              if prune == False and len(temp) == k:
                  augmented = True
                  temp.sort()
                  complexID = ','.join(temp)
                  if not (complexID in currComplexes):
                      complexMembers = complexID.split(',')
                      scoresPerPair = []
                      for x in range((len(complexMembers) - 1)):
                          for y in range(x + 1, len(complexMembers)):
                              pairF = (complexMembers[x], complexMembers[y])
                              pairB = (complexMembers[y], complexMembers[x])
                              pairScore = 0.0
                              if pairF in comparisonMatrix:
                                  pairScore = comparisonMatrix[pairF]
                              elif pairB in comparisonMatrix:
                                  pairScore = comparisonMatrix[pairB]
                              scoresPerPair.append(pairScore)
                      newWeight = min(scoresPerPair)
                      if newWeight < TOP_CUTOFF:
                          continue
                      currComplexes[complexID] = (temp, newWeight)
                      for prot in temp:
                          if not (prot in restrictedProteins):
                              restrictedProteins[prot] = prot
                      scores[complexID] = newWeight
          if not augmented:
              candidateComplex.sort()
              key = ','.join(candidateComplex)
              outline = key + '\t' + str(weight) + '\n'
              outfilePredicted.writelines(outline)
              complexesCount += 1

      #Close outfile
      outfilePredicted.close()

      outline = '\t Total of predicted ' + str(k-1) + '-complexes'
      print(outline, complexesCount, len(candidateComplexes))

      outline = 'Candidate ' + str(k) + '-complexes'
      if len(scores) < 1:
          print(outline, len(currComplexes))
          continue
      averageWeight = np.mean(list(scores.values()))
      stdWeight = np.std(list(scores.values()))
      weightThreshold = averageWeight + (1.0 * stdWeight)

      print(outline, len(currComplexes), averageWeight, stdWeight, weightThreshold)

      k = k + 1
  '''

# of nodes 1571
# of edges 13457
52


##get all cliques

In [ ]:
def main(argv):
    if (len(argv) == 3):
        graphFilename = argv[1]
        outfilenamePrefix = argv[2]
    else:
        print ("<<ERROR>> Invalid number of parameters!")
        return
   
    infile  = open(graphFilename, "r")
    
    headers = infile.readline().strip()
   
    G = nx.Graph()
    comparisonMatrix = {}
    for line in infile:
        line = line.strip()
        edgeType, prot1ID, prot2ID, pScore = line.split(' ')
        G.add_edge(prot1ID, prot2ID)  
        proteinPair = (prot1ID, prot2ID)
        comparisonMatrix[proteinPair] = float(pScore)
    #Close input file    
    infile.close()
    
    print ("# of nodes", len(G.nodes))
    print ("# of edges", len(G.edges))
    
    cliques = nx.find_cliques(G)

    predictedComplexes = {}
    for cliqueVerticesList in cliques:
        cliqueVerticesList.sort()
        complexID = ','.join(cliqueVerticesList)
        complexSize = len(cliqueVerticesList)
        scoresPerPair = []
        for i in range((len(cliqueVerticesList) - 1)):
            for j in range(i + 1, len(cliqueVerticesList)):
                pairF = (cliqueVerticesList[i], cliqueVerticesList[j])
                pairB = (cliqueVerticesList[j], cliqueVerticesList[i])
                pairScore = 0.0
                if pairF in comparisonMatrix:
                    pairScore = comparisonMatrix[pairF]
                elif pairB in comparisonMatrix:
                    pairScore = comparisonMatrix[pairB]
                scoresPerPair.append(pairScore)
        complexScore = min(scoresPerPair)
        if complexSize in predictedComplexes:
            predictedComplexes[complexSize].append((complexID, complexScore))
        else:
            predictedComplexes[complexSize] = [(complexID, complexScore)]

    print (len(predictedComplexes.keys()))

    for complexSize, complexesSet in predictedComplexes.items():
        outfilename = outfilenamePrefix + str(complexSize) + '.txt'
        outfile  = open(outfilename, "w")
        for complexID, complexScore in complexesSet:
            outline = complexID + '\t' + str(complexScore) + '\n'
            outfile.writelines(outline)
        #Close output file    
        outfile.close()
    
    return       

##    'integrated_MM+TSCS+Exp+Abun+Loc_candidates_complexesSize_2_top001.graph', 'integrated_MM+TSCS+Exp+Abun+Loc_features_all_pairs_model_either_predictions_rf_predicted_complexesSize_'])

##gw modifications: body moved outside of function, filename structure changes

In [ ]:
for cell in cellLines:
  pairsInfoFilename = modelPerformances_path + expName + '/' + cell + pairsInfo_fileSuffix
  pairsPredictionsFilename = modelPerformances_path + expName + '/' + cell + pairsScores_fileSuffix

  TOP_CUTOFF = cutoff   

  #Open input file
  try:
      infilePairsInfo = open(pairsInfoFilename, "r")
  except IOError as e:
      print("<<ERROR>> Unable to open the file", pairsInfoFilename, "\nThis program will be quiting now."), e
      sys.exit()

  #Open input file
  predictionsFilename = pairsPredictionsFilename + '.txt'
  try:
      infileScores = open(predictionsFilename, "r")
  except IOError as e:
      print("<<ERROR>> Unable to open the file", predictionsFilename, "\nThis program will be quiting now."), e
      sys.exit()

  proteins = {}
  restrictedProteins = {}
  currComplexes = {}
  scores = {}
  comparisonMatrix = {}
  outfilename = pairsPredictionsFilename + '_candidates_complexesSize_2.txt'
  outfile = open(outfilename, 'w')
  for line in infileScores:
      line = line.strip()
      tokens = line.split('\t')
      pScore = float(tokens[0])

      infoLine = infilePairsInfo.readline().strip()
      tokens = infoLine.split('\t')
      classLabel = tokens[0]
      prot1ID = tokens[1]
      prot2ID = tokens[2]
      pair1 = (prot1ID, prot2ID)
      pair2 = (prot2ID, prot1ID)
      comparisonMatrix[pair1] = pScore

      if not (prot1ID in proteins):
          proteins[prot1ID] = prot1ID
      if not (prot2ID in proteins):
          proteins[prot2ID] = prot2ID

      temp = []
      if pScore >= TOP_CUTOFF:
          if not (prot1ID in restrictedProteins):
              restrictedProteins[prot1ID] = prot1ID
          if not (prot2ID in restrictedProteins):
              restrictedProteins[prot2ID] = prot2ID

          if not (prot1ID in temp):
              temp.append(prot1ID)
          if not (prot2ID in temp):
              temp.append(prot2ID)
          temp.sort()
          complexID = ','.join(temp)
          if not (complexID in currComplexes):
              outline = complexID + '\t' + str(pScore) + '\n'
              outfile.writelines(outline)
              currComplexes[complexID] = (temp, pScore)
              scores[complexID] = pScore

  #Close files
  infilePairsInfo.close()
  infileScores.close()
  outfile.close()
  outfilename = pairsPredictionsFilename.split('_features_')[0] + '_candidates_complexesSize_2@' + str(cutoff) + '.gv'
  outfile = open(outfilename, 'w')

  #Prepare headers
  outline = 'graph {' + '\n'
  outline += '\t node [shape = circle];' + '\n'
  outline += '\t edge [style = solid];' + '\n'
  outfile.writelines(outline)
  nodeID = 0
  protID2nodeID = {}
  for protID in restrictedProteins.keys():
      if not (protID in protID2nodeID):
          protID2nodeID[protID] = nodeID
          outline = '\t n' + str(nodeID) + ' [label = p' + str(nodeID) + '];' + '\n'
          outfile.writelines(outline)
          nodeID += 1

  outfilenameGraph = pairsPredictionsFilename.split('_features_')[0] + '_candidates_complexesSize_2@' + str(cutoff) + '.graph'
  outfileGraph = open(outfilenameGraph, 'w')
  outlineGraph = 'p edge ' + str(len(restrictedProteins)) + '\n'
  outfileGraph.writelines(outlineGraph)
  for complexID, attributes in currComplexes.items():
      prot1ID, prot2ID = complexID.split(',')
      edgeScore = copy.copy(attributes[1])
      if prot1ID in protID2nodeID:
          node1ID = protID2nodeID[prot1ID]
      if prot2ID in protID2nodeID:
          node2ID = protID2nodeID[prot2ID]
      outline = '\t n' + str(node1ID) + ' -- ' + 'n' + str(node2ID) + ' [label = ' + str(edgeScore) + '];' + '\n'
      outfile.writelines(outline)
      outlineGraph = 'e ' + str(prot1ID) + ' ' + str(prot2ID) + ' ' + str(edgeScore) + '\n'
      outfileGraph.writelines(outlineGraph)

      #Prepare footers
      outline = '}' + '\n'
  outfile.writelines(outline)

  #Close file
  outfile.close()
  outfileGraph.close()

  percentile = np.percentile(list(scores.values()), targetPercentile)
  averageWeight = np.mean(list(scores.values()))
  stdWeight = np.std(list(scores.values()))
  ##weightThreshold = averageWeight + (1.0 * stdWeight)
  ##weightThreshold = averageWeight + (2.0 * stdWeight)
  weightThreshold = percentile

  print('# of proteins', len(proteins), len(restrictedProteins))
  print('Candidate pairwise interactions', len(comparisonMatrix))
  print('\t High-scoring pairwise interactions', len(currComplexes))
  print('\t Average pairwise score', averageWeight)
  print('\t Standard deviation of pairwise scores', stdWeight)
  outline = '\t ' + str(targetPercentile) + ' percentile score ' + str(weightThreshold)
  print(outline)

  k = 3
  while len(currComplexes) > 0 and k <= MAX_K:
      complexesCount = 0
      candidateComplexes = copy.copy(currComplexes)
      print (k - 1), len(restrictedProteins)
      ##proteinSet = proteins.keys()
      proteinSet = restrictedProteins.keys()
      restrictedProteins = {}
      scores = {}
      currComplexes = {}
      alreadyPrinted = {}
      ##outfilenameCandidates = outputPATH + pairsPredictionsFilename + '_candidates_complexesSize_' + str(k) + '.txt'
      ##outfileCandidates = open(outfilenameCandidates, 'w')
      outfilenamePredicted = pairsPredictionsFilename + '_predicted_complexesSize_' + str(k-1) + '.txt'
      outfilePredicted = open(outfilenamePredicted, "w")
      for candidateComplex, weight in candidateComplexes.values():
          ##prune = False
          augmented = False
          ##temp = []
          ####temp = copy.copy(candidateComplex)
          ####if weight < TOP_CUTOFF:
          ####continue
          for prot2ID in proteinSet:
              prune = False
              temp = []
              for i in range(len(candidateComplex)):
                  prot1ID = candidateComplex[i]
                  if prot2ID in candidateComplex:
                      break
                  pair1 = (prot1ID, prot2ID)
                  pair2 = (prot2ID, prot1ID)
                  if pair1 in comparisonMatrix:
                      pScore = comparisonMatrix[pair1]
                  elif pair2 in comparisonMatrix:
                      pScore = comparisonMatrix[pair2]
                  else:
                      break
                  if pScore >= TOP_CUTOFF: #PAIRWISE_INTERACTIONS_CUTOFF:
                      if not (prot1ID in temp):
                          temp.append(prot1ID)
                      if not (prot2ID in temp):
                          temp.append(prot2ID)
                  else:
                      prune = True
                      break
              if prune == False and len(temp) == k:
                  augmented = True
                  temp.sort()
                  complexID = ','.join(temp)
                  if not (complexID in currComplexes):
                      complexMembers = complexID.split(',')
                      scoresPerPair = []
                      for x in range((len(complexMembers) - 1)):
                          for y in range(x + 1, len(complexMembers)):
                              pairF = (complexMembers[x], complexMembers[y])
                              pairB = (complexMembers[y], complexMembers[x])
                              pairScore = 0.0
                              if pairF in comparisonMatrix:
                                  pairScore = comparisonMatrix[pairF]
                              elif pairB in comparisonMatrix:
                                  pairScore = comparisonMatrix[pairB]
                              scoresPerPair.append(pairScore)
                      newWeight = min(scoresPerPair)
                      ####newWeight = np.mean(scoresPerPair)
                      ####newWeight = np.percentile(scoresPerPair, 50)
                      if newWeight < TOP_CUTOFF:
                          continue
                          ##outline = complexID
                      ##for pairScore in scoresPerPair:
                          ##outline += '\t' + str(pairScore)
                      ##outline += '\n'
                      ##outfileCandidates.writelines(outline)
                      currComplexes[complexID] = (temp, newWeight)
                      for prot in temp:
                          if not (prot in restrictedProteins):
                              restrictedProteins[prot] = prot
                      scores[complexID] = newWeight
          if not augmented:
              candidateComplex.sort()
              key = ','.join(candidateComplex)
              outline = key + '\t' + str(weight) + '\n'
              outfilePredicted.writelines(outline)
              complexesCount += 1

      #Close outfile
      ##outfileCandidates.close()
      outfilePredicted.close()

      outline = '\t Total of predicted ' + str(k-1) + '-complexes'
      print(outline, complexesCount, len(candidateComplexes))

      outline = 'Candidate ' + str(k) + '-complexes'
      if len(scores) < 1:
          print(outline, len(currComplexes))
          continue
      ##percentile = np.percentile(list(scores.values()), targetPercentile)
      averageWeight = np.mean(list(scores.values()))
      stdWeight = np.std(list(scores.values()))
      weightThreshold = averageWeight + (1.0 * stdWeight)
      ##weightThreshold = averageWeight + (2.0 * stdWeight)
      ##weightThreshold = percentile

      print(outline, len(currComplexes), averageWeight, stdWeight, weightThreshold)

      k = k + 1

###gw modifications: filename structure changes

In [ ]:
def buildComplexes(pairsInfoFilename, pairsPredictionsFilename, cutoff, targetPercentile):

  TOP_CUTOFF = cutoff   #0.7740 for 0.05% high-scoring pairs; 0.8663 for 0.02% high-scoring pairs
                        #0.9251 for 0.01% high-scoring pairs; 0.9615 for 0.005% high-scoring pairs
                        #0.9251 for 0.02% high-scoring pairs; 0. for 0.005% high-scoring pairs
                        #0.9840 for 0.002% high-scoring pairs; 0.9898 for 0.001% high-scoring pairs

  #Open input file
  try:
      infilePairsInfo = open(pairsInfoFilename, "r")
  except IOError as e:
      print("<<ERROR>> Unable to open the file", pairsInfoFilename, "\nThis program will be quiting now."), e
      sys.exit()

  #Open input file
  predictionsFilename = pairsPredictionsFilename + '.txt'
  try:
      infileScores = open(predictionsFilename, "r")
  except IOError as e:
      print("<<ERROR>> Unable to open the file", predictionsFilename, "\nThis program will be quiting now."), e
      sys.exit()

  proteins = {}
  restrictedProteins = {}
  currComplexes = {}
  scores = {}
  comparisonMatrix = {}
  outfilename = pairsPredictionsFilename + '_candidates_complexesSize_2.txt'
  outfile = open(outfilename, 'w')
  for line in infileScores:
      line = line.strip()
      tokens = line.split('\t')
      pScore = float(tokens[0])

      infoLine = infilePairsInfo.readline().strip()
      tokens = infoLine.split('\t')
      classLabel = tokens[0]
      prot1ID = tokens[1]
      prot2ID = tokens[2]
      pair1 = (prot1ID, prot2ID)
      pair2 = (prot2ID, prot1ID)
      comparisonMatrix[pair1] = pScore

      if not (prot1ID in proteins):
          proteins[prot1ID] = prot1ID
      if not (prot2ID in proteins):
          proteins[prot2ID] = prot2ID

      temp = []
      if pScore >= TOP_CUTOFF:
          if not (prot1ID in restrictedProteins):
              restrictedProteins[prot1ID] = prot1ID
          if not (prot2ID in restrictedProteins):
              restrictedProteins[prot2ID] = prot2ID

          if not (prot1ID in temp):
              temp.append(prot1ID)
          if not (prot2ID in temp):
              temp.append(prot2ID)
          temp.sort()
          complexID = ','.join(temp)
          if not (complexID in currComplexes):
              outline = complexID + '\t' + str(pScore) + '\n'
              outfile.writelines(outline)
              currComplexes[complexID] = (temp, pScore)
              scores[complexID] = pScore

  #Close files
  infilePairsInfo.close()
  infileScores.close()
  outfile.close()
  outfilename = pairsPredictionsFilename.split('_features_')[0] + '_candidates_complexesSize_2@' + str(cutoff) + '.gv'
  outfile = open(outfilename, 'w')

  #Prepare headers
  outline = 'graph {' + '\n'
  outline += '\t node [shape = circle];' + '\n'
  outline += '\t edge [style = solid];' + '\n'
  outfile.writelines(outline)
  nodeID = 0
  protID2nodeID = {}
  for protID in restrictedProteins.keys():
      if not (protID in protID2nodeID):
          protID2nodeID[protID] = nodeID
          outline = '\t n' + str(nodeID) + ' [label = p' + str(nodeID) + '];' + '\n'
          outfile.writelines(outline)
          nodeID += 1

  outfilenameGraph = pairsPredictionsFilename.split('_features_')[0] + '_candidates_complexesSize_2@' + str(cutoff) + '.graph'
  outfileGraph = open(outfilenameGraph, 'w')
  outlineGraph = 'p edge ' + str(len(restrictedProteins)) + '\n'
  outfileGraph.writelines(outlineGraph)
  for complexID, attributes in currComplexes.items():
      prot1ID, prot2ID = complexID.split(',')
      edgeScore = copy.copy(attributes[1])
      if prot1ID in protID2nodeID:
          node1ID = protID2nodeID[prot1ID]
      if prot2ID in protID2nodeID:
          node2ID = protID2nodeID[prot2ID]
      outline = '\t n' + str(node1ID) + ' -- ' + 'n' + str(node2ID) + ' [label = ' + str(edgeScore) + '];' + '\n'
      outfile.writelines(outline)
      outlineGraph = 'e ' + str(prot1ID) + ' ' + str(prot2ID) + ' ' + str(edgeScore) + '\n'
      outfileGraph.writelines(outlineGraph)

      #Prepare footers
      outline = '}' + '\n'
  outfile.writelines(outline)

  #Close file
  outfile.close()
  outfileGraph.close()

  #return

  percentile = np.percentile(list(scores.values()), targetPercentile)
  averageWeight = np.mean(list(scores.values()))
  stdWeight = np.std(list(scores.values()))
  ##weightThreshold = averageWeight + (1.0 * stdWeight)
  ##weightThreshold = averageWeight + (2.0 * stdWeight)
  weightThreshold = percentile

  print('# of proteins', len(proteins), len(restrictedProteins))
  print('Candidate pairwise interactions', len(comparisonMatrix))
  print('\t High-scoring pairwise interactions', len(currComplexes))
  print('\t Average pairwise score', averageWeight)
  print('\t Standard deviation of pairwise scores', stdWeight)
  outline = '\t ' + str(targetPercentile) + ' percentile score ' + str(weightThreshold)
  print(outline)

  k = 3
  while len(currComplexes) > 0 and k <= MAX_K:
      complexesCount = 0
      candidateComplexes = copy.copy(currComplexes)
      print (k - 1), len(restrictedProteins)
      ##proteinSet = proteins.keys()
      proteinSet = restrictedProteins.keys()
      restrictedProteins = {}
      scores = {}
      currComplexes = {}
      alreadyPrinted = {}
      ##outfilenameCandidates = outputPATH + pairsPredictionsFilename + '_candidates_complexesSize_' + str(k) + '.txt'
      ##outfileCandidates = open(outfilenameCandidates, 'w')
      outfilenamePredicted = pairsPredictionsFilename + '_predicted_complexesSize_' + str(k-1) + '.txt'
      outfilePredicted = open(outfilenamePredicted, "w")
      for candidateComplex, weight in candidateComplexes.values():
          ##prune = False
          augmented = False
          ##temp = []
          ####temp = copy.copy(candidateComplex)
          ####if weight < TOP_CUTOFF:
          ####continue
          for prot2ID in proteinSet:
              prune = False
              temp = []
              for i in range(len(candidateComplex)):
                  prot1ID = candidateComplex[i]
                  if prot2ID in candidateComplex:
                      break
                  pair1 = (prot1ID, prot2ID)
                  pair2 = (prot2ID, prot1ID)
                  if pair1 in comparisonMatrix:
                      pScore = comparisonMatrix[pair1]
                  elif pair2 in comparisonMatrix:
                      pScore = comparisonMatrix[pair2]
                  else:
                      break
                  if pScore >= TOP_CUTOFF: #PAIRWISE_INTERACTIONS_CUTOFF:
                      if not (prot1ID in temp):
                          temp.append(prot1ID)
                      if not (prot2ID in temp):
                          temp.append(prot2ID)
                  else:
                      prune = True
                      break
              if prune == False and len(temp) == k:
                  augmented = True
                  temp.sort()
                  complexID = ','.join(temp)
                  if not (complexID in currComplexes):
                      complexMembers = complexID.split(',')
                      scoresPerPair = []
                      for x in range((len(complexMembers) - 1)):
                          for y in range(x + 1, len(complexMembers)):
                              pairF = (complexMembers[x], complexMembers[y])
                              pairB = (complexMembers[y], complexMembers[x])
                              pairScore = 0.0
                              if pairF in comparisonMatrix:
                                  pairScore = comparisonMatrix[pairF]
                              elif pairB in comparisonMatrix:
                                  pairScore = comparisonMatrix[pairB]
                              scoresPerPair.append(pairScore)
                      newWeight = min(scoresPerPair)
                      ####newWeight = np.mean(scoresPerPair)
                      ####newWeight = np.percentile(scoresPerPair, 50)
                      if newWeight < TOP_CUTOFF:
                          continue
                          ##outline = complexID
                      ##for pairScore in scoresPerPair:
                          ##outline += '\t' + str(pairScore)
                      ##outline += '\n'
                      ##outfileCandidates.writelines(outline)
                      currComplexes[complexID] = (temp, newWeight)
                      for prot in temp:
                          if not (prot in restrictedProteins):
                              restrictedProteins[prot] = prot
                      scores[complexID] = newWeight
          if not augmented:
              candidateComplex.sort()
              key = ','.join(candidateComplex)
              outline = key + '\t' + str(weight) + '\n'
              outfilePredicted.writelines(outline)
              complexesCount += 1

      #Close outfile
      ##outfileCandidates.close()
      outfilePredicted.close()

      outline = '\t Total of predicted ' + str(k-1) + '-complexes'
      print(outline, complexesCount, len(candidateComplexes))

      outline = 'Candidate ' + str(k) + '-complexes'
      if len(scores) < 1:
          print(outline, len(currComplexes))
          continue
      ##percentile = np.percentile(list(scores.values()), targetPercentile)
      averageWeight = np.mean(list(scores.values()))
      stdWeight = np.std(list(scores.values()))
      weightThreshold = averageWeight + (1.0 * stdWeight)
      ##weightThreshold = averageWeight + (2.0 * stdWeight)
      ##weightThreshold = percentile

      print(outline, len(currComplexes), averageWeight, stdWeight, weightThreshold)

      k = k + 1

  return

In [ ]:
def main(argv):
    if len(argv) == 5:
        pairsInfoFilename = argv[1]
        pairsPredictionsFilename = argv[2]
        cutoff = float(argv[3])
        targetPercentile = float(argv[4])
    else:
        print("<<ERROR>> Invalid number of parameters!")
        return

    buildComplexes(pairsInfoFilename, pairsPredictionsFilename, cutoff, targetPercentile)

###gw modifications: filename structure changes, comment "polishing"

In [ ]:
def buildComplexes(pairsInfoFilename, pairsPredictionsFilename, cutoff, targetPercentile, outputPATH):

  MAX_K = 45
  TOP_CUTOFF = cutoff

  #Open input file
  try:
      infilePairsInfo = open(pairsInfoFilename, "r")
  except IOError as e:
      print("<<ERROR>> Unable to open the file", pairsInfoFilename, "\nThis program will be quiting now."), e
      sys.exit()

  #Open input file
  predictionsFilename = pairsPredictionsFilename + '.txt'
  try:
      infileScores = open(predictionsFilename, "r")
  except IOError as e:
      print("<<ERROR>> Unable to open the file", predictionsFilename, "\nThis program will be quiting now."), e
      sys.exit()

  proteins = {}
  restrictedProteins = {}
  currComplexes = {}
  scores = {}
  comparisonMatrix = {}
  outfilename = pairsPredictionsFilename + '_candidates_complexesSize_2.txt'
  outfile = open(outfilename, 'w')
  for line in infileScores:
      line = line.strip()
      tokens = line.split('\t')
      pScore = float(tokens[0])

      infoLine = infilePairsInfo.readline().strip()
      tokens = infoLine.split('\t')
      classLabel = tokens[0]
      prot1ID = tokens[1]
      prot2ID = tokens[2]
      pair1 = (prot1ID, prot2ID)
      pair2 = (prot2ID, prot1ID)
      comparisonMatrix[pair1] = pScore

      if not (prot1ID in proteins):
          proteins[prot1ID] = prot1ID
      if not (prot2ID in proteins):
          proteins[prot2ID] = prot2ID

      temp = []
      if pScore >= TOP_CUTOFF:
          if not (prot1ID in restrictedProteins):
              restrictedProteins[prot1ID] = prot1ID
          if not (prot2ID in restrictedProteins):
              restrictedProteins[prot2ID] = prot2ID

          if not (prot1ID in temp):
              temp.append(prot1ID)
          if not (prot2ID in temp):
              temp.append(prot2ID)
          temp.sort()
          complexID = ','.join(temp)
          if not (complexID in currComplexes):
              outline = complexID + '\t' + str(pScore) + '\n'
              outfile.writelines(outline)
              currComplexes[complexID] = (temp, pScore)
              scores[complexID] = pScore

  #Close files
  infilePairsInfo.close()
  infileScores.close()
  outfile.close()
  outfilename = pairsPredictionsFilename.split('_features_')[0] + '_candidates_complexesSize_2@' + str(cutoff) + '.gv'
  outfile = open(outfilename, 'w')

  #Prepare headers
  outline = 'graph {' + '\n'
  outline += '\t node [shape = circle];' + '\n'
  outline += '\t edge [style = solid];' + '\n'
  outfile.writelines(outline)
  nodeID = 0
  protID2nodeID = {}
  for protID in restrictedProteins.keys():
      if not (protID in protID2nodeID):
          protID2nodeID[protID] = nodeID
          outline = '\t n' + str(nodeID) + ' [label = p' + str(nodeID) + '];' + '\n'
          outfile.writelines(outline)
          nodeID += 1

  outfilenameGraph = pairsPredictionsFilename.split('_features_')[0] + '_candidates_complexesSize_2@' + str(cutoff) + '.graph'
  outfileGraph = open(outfilenameGraph, 'w')
  outlineGraph = 'p edge ' + str(len(restrictedProteins)) + '\n'
  outfileGraph.writelines(outlineGraph)
  for complexID, attributes in currComplexes.items():
      prot1ID, prot2ID = complexID.split(',')
      edgeScore = copy.copy(attributes[1])
      if prot1ID in protID2nodeID:
          node1ID = protID2nodeID[prot1ID]
      if prot2ID in protID2nodeID:
          node2ID = protID2nodeID[prot2ID]
      outline = '\t n' + str(node1ID) + ' -- ' + 'n' + str(node2ID) + ' [label = ' + str(edgeScore) + '];' + '\n'
      outfile.writelines(outline)
      outlineGraph = 'e ' + str(prot1ID) + ' ' + str(prot2ID) + ' ' + str(edgeScore) + '\n'
      outfileGraph.writelines(outlineGraph)

      #Prepare footers
      outline = '}' + '\n'
  outfile.writelines(outline)

  #Close file
  outfile.close()
  outfileGraph.close()

  percentile = np.percentile(list(scores.values()), targetPercentile)
  averageWeight = np.mean(list(scores.values()))
  stdWeight = np.std(list(scores.values()))
  weightThreshold = percentile

  print('# of proteins', len(proteins), len(restrictedProteins))
  print('Candidate pairwise interactions', len(comparisonMatrix))
  print('\t High-scoring pairwise interactions', len(currComplexes))
  print('\t Average pairwise score', averageWeight)
  print('\t Standard deviation of pairwise scores', stdWeight)
  outline = '\t ' + str(targetPercentile) + ' percentile score ' + str(weightThreshold)
  print(outline)

  k = 3
  while len(currComplexes) > 0 and k <= MAX_K:
      complexesCount = 0
      candidateComplexes = copy.copy(currComplexes)
      print (k - 1), len(restrictedProteins)
      proteinSet = restrictedProteins.keys()
      restrictedProteins = {}
      scores = {}
      currComplexes = {}
      alreadyPrinted = {}
      outfilenamePredicted = pairsPredictionsFilename + '_predicted_complexesSize_' + str(k-1) + '.txt'
      outfilePredicted = open(outfilenamePredicted, "w")
      for candidateComplex, weight in candidateComplexes.values():
          augmented = False
          for prot2ID in proteinSet:
              prune = False
              temp = []
              for i in range(len(candidateComplex)):
                  prot1ID = candidateComplex[i]
                  if prot2ID in candidateComplex:
                      break
                  pair1 = (prot1ID, prot2ID)
                  pair2 = (prot2ID, prot1ID)
                  if pair1 in comparisonMatrix:
                      pScore = comparisonMatrix[pair1]
                  elif pair2 in comparisonMatrix:
                      pScore = comparisonMatrix[pair2]
                  else:
                      break
                  if pScore >= TOP_CUTOFF: #PAIRWISE_INTERACTIONS_CUTOFF:
                      if not (prot1ID in temp):
                          temp.append(prot1ID)
                      if not (prot2ID in temp):
                          temp.append(prot2ID)
                  else:
                      prune = True
                      break
              if prune == False and len(temp) == k:
                  augmented = True
                  temp.sort()
                  complexID = ','.join(temp)
                  if not (complexID in currComplexes):
                      complexMembers = complexID.split(',')
                      scoresPerPair = []
                      for x in range((len(complexMembers) - 1)):
                          for y in range(x + 1, len(complexMembers)):
                              pairF = (complexMembers[x], complexMembers[y])
                              pairB = (complexMembers[y], complexMembers[x])
                              pairScore = 0.0
                              if pairF in comparisonMatrix:
                                  pairScore = comparisonMatrix[pairF]
                              elif pairB in comparisonMatrix:
                                  pairScore = comparisonMatrix[pairB]
                              scoresPerPair.append(pairScore)
                      newWeight = min(scoresPerPair)
                      if newWeight < TOP_CUTOFF:
                          continue

                      currComplexes[complexID] = (temp, newWeight)
                      for prot in temp:
                          if not (prot in restrictedProteins):
                              restrictedProteins[prot] = prot
                      scores[complexID] = newWeight
          if not augmented:
              candidateComplex.sort()
              key = ','.join(candidateComplex)
              outline = key + '\t' + str(weight) + '\n'
              outfilePredicted.writelines(outline)
              complexesCount += 1

      #Close outfile
      outfilePredicted.close()

      outline = '\t Total of predicted ' + str(k-1) + '-complexes'
      print(outline, complexesCount, len(candidateComplexes))
      outline = 'Candidate ' + str(k) + '-complexes'
      if len(scores) < 1:
          print(outline, len(currComplexes))
          continue
      averageWeight = np.mean(list(scores.values()))
      stdWeight = np.std(list(scores.values()))
      weightThreshold = averageWeight + (1.0 * stdWeight)
      print(outline, len(currComplexes), averageWeight, stdWeight, weightThreshold)

      k = k + 1

  return


def main(argv):
    if len(argv) == 6:
        pairsInfoFilename = argv[1]
        pairsPredictionsFilename = argv[2]
        cutoff = float(argv[3])
        targetPercentile = float(argv[4])
        outputPATH = argv[5]
    else:
        print("<<ERROR>> Invalid number of parameters!")
        return

    buildComplexes(pairsInfoFilename, pairsPredictionsFilename, cutoff, targetPercentile, outputPATH)


if __name__ == '__main__':
  #main(sys.args) #Alternate call, if you are using IDLE.
  #main(['buildComplexesFromPairwiseScoresRecursive.py', 'all_either_model.tsv', 'MM+TSCS+Exp+Abun+Loc+Kaggle_features_all_either_model_predictions_rf_400', '0.760267', '50.0', './results/top02_min/'])

#build complexes from top 95% STRING complexes

##prep STRING data for complex building

In [ ]:
STRING_protLinks_detailedChannel_path = workDir + 'proteinComplexes/9606.protein.links.detailed.v11.5.txt'

In [ ]:
STRING_protLinks_detailedChannel = pd.read_csv(STRING_protLinks_detailedChannel_path, sep=' ')

In [ ]:
STRING_protLinks_detailedChannel.head()

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
0,9606.ENSP00000000233,9606.ENSP00000379496,0.0,0.0,0.0,54.0,0.0,0.0,144.0,155.0
1,9606.ENSP00000000233,9606.ENSP00000314067,0.0,0.0,0.0,0.0,180.0,0.0,61.0,197.0
2,9606.ENSP00000000233,9606.ENSP00000263116,0.0,0.0,0.0,62.0,152.0,0.0,101.0,222.0
3,9606.ENSP00000000233,9606.ENSP00000361263,0.0,0.0,0.0,0.0,161.0,0.0,64.0,181.0
4,9606.ENSP00000000233,9606.ENSP00000409666,0.0,0.0,0.0,82.0,213.0,0.0,72.0,270.0


In [ ]:
STRING_protLinks_detailedChannel_justIDs = STRING_protLinks_detailedChannel.protein1.append(STRING_protLinks_detailedChannel_justIDs.protein2)

In [ ]:
STRING_protLinks_detailedChannel_justIDs.shape

(8341540,)

In [ ]:
STRING_protLinks_detailedChannel_justIDs_unique = STRING_protLinks_detailedChannel_justIDs.drop_duplicates()

In [ ]:
STRING_protLinks_detailedChannel_justIDs_unique.shape

(19368,)

In [ ]:
STRING_protLinks_detailedChannel_justIDs_unique.dropna(inplace=True)

In [ ]:
STRING_protLinks_detailedChannel_justIDs_unique

0          9606.ENSP00000000233
1440       9606.ENSP00000000412
1920       9606.ENSP00000001008
3620       9606.ENSP00000001146
4370       9606.ENSP00000002125
                   ...         
3742559    9606.ENSP00000436891
3776661    9606.ENSP00000477724
4002611    9606.ENSP00000485043
4018559    9606.ENSP00000404583
4041929    9606.ENSP00000485142
Length: 19367, dtype: object

In [ ]:
STRING_protLinks_detailedChannel_justIDs_unique.to_csv(workDir + 'proteinComplexes/STRING_fullComplement_geneID.tsv', sep='\t', index=False, header=False)

In [ ]:
STRING_GeneID_convert_complementSubset = pd.read_csv(workDir + 'proteinComplexes/STRING_to_GeneID.tab', sep='\t')

In [ ]:
STRING_GeneID_convert_complementSubset.head()

,Entry,Entry name,Protein names,Gene names,Cross-reference (STRING),Cross-reference (GeneID)
0,P84085,ARF5_HUMAN,ADP-ribosylation factor 5,ARF5,9606.ENSP00000000233,381
1,P20645,MPRD_HUMAN,Cation-dependent mannose-6-phosphate receptor ...,M6PR MPR46 MPRD,9606.ENSP00000000412,4074
2,Q02790,FKBP4_HUMAN,Peptidyl-prolyl cis-trans isomerase FKBP4 (PPI...,FKBP4 FKBP52,9606.ENSP00000001008,2288
3,Q9NR63,CP26B_HUMAN,Cytochrome P450 26B1 (EC 1.14.13.-) (Cytochrom...,CYP26B1 CYP26A2 P450RAI2,9606.ENSP00000001146,56603
4,Q7L592,NDUF7_HUMAN,"Protein arginine methyltransferase NDUFAF7, mi...",NDUFAF7 C2orf56 PRO1853,9606.ENSP00000002125,55471


In [ ]:
STRING_GeneID_convert_complementSubset['Cross-reference (GeneID)'] = STRING_GeneID_convert_complementSubset['Cross-reference (GeneID)'].str.replace(';','')

In [ ]:
STRING_GeneID_convert_complementSubset['Cross-reference (STRING)'] = STRING_GeneID_convert_complementSubset['Cross-reference (STRING)'].str.replace(';','')

In [ ]:
STRING_GeneID_convert_complementSubset.shape

(18869, 6)

In [ ]:
geneID_stringID_converter = dict(zip(STRING_GeneID_convert_complementSubset['Cross-reference (STRING)'], STRING_GeneID_convert_complementSubset['Cross-reference (GeneID)']))

In [ ]:
STRING_protLinks_detailedChannel['protein1_geneID_alias'] = STRING_protLinks_detailedChannel.protein1.map(geneID_stringID_converter)
STRING_protLinks_detailedChannel['protein2_geneID_alias'] = STRING_protLinks_detailedChannel.protein2.map(geneID_stringID_converter)

In [ ]:
STRING_protLinks_detailedChannel

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,protein1_geneID_alias,protein2_geneID_alias
0,9606.ENSP00000000233,9606.ENSP00000379496,0.0,0.0,0.0,54.0,0.0,0.0,144.0,155.0,381,5137
1,9606.ENSP00000000233,9606.ENSP00000314067,0.0,0.0,0.0,0.0,180.0,0.0,61.0,197.0,381,5062
2,9606.ENSP00000000233,9606.ENSP00000263116,0.0,0.0,0.0,62.0,152.0,0.0,101.0,222.0,381,9609
3,9606.ENSP00000000233,9606.ENSP00000361263,0.0,0.0,0.0,0.0,161.0,0.0,64.0,181.0,381,2889
4,9606.ENSP00000000233,9606.ENSP00000409666,0.0,0.0,0.0,82.0,213.0,0.0,72.0,270.0,381,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4170765,9606.ENSP00000311453,9606.ENSP00000348573,0.0,0.0,0.0,0.0,0.0,0.0,216.0,216.0,5151,10142
4170766,9606.ENSP00000311453,9606.ENSP00000485525,0.0,0.0,0.0,0.0,0.0,900.0,0.0,900.0,5151,158
4170767,9606.ENSP00000311453,9606.ENSP00000345974,0.0,0.0,0.0,0.0,0.0,0.0,307.0,307.0,5151,55556
4170768,9606.ENSP00000311453,9606.ENSP00000263285,0.0,0.0,0.0,0.0,0.0,0.0,151.0,150.0,5151,4063


In [ ]:
STRING_protLinks_detailedChannel_complementSubset = STRING_protLinks_detailedChannel.dropna(subset=['protein1_geneID_alias', 'protein2_geneID_alias'])

In [ ]:
STRING_protLinks_detailedChannel_complementSubset.shape

(4002102, 12)

In [ ]:
numProteins_wo_geneIDs = len(STRING_protLinks_detailedChannel) - len(STRING_protLinks_detailedChannel_complementSubset)
print(numProteins_wo_geneIDs)
print(numProteins_wo_geneIDs/len(STRING_protLinks_detailedChannel))

168668
0.04044049420131055


In [ ]:
combinedScores = STRING_protLinks_detailedChannel_complementSubset.combined_score.to_numpy().reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
combinedScores_scaled = min_max_scaler.fit_transform(combinedScores)
STRING_protLinks_detailedChannel_complementSubset['combined_score_scaled'] = combinedScores_scaled

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
STRING_protLinks_detailedChannel_complementSubset.head()

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,protein1_geneID_alias,protein2_geneID_alias,combined_score_scaled
0,9606.ENSP00000000233,9606.ENSP00000379496,0.0,0.0,0.0,54.0,0.0,0.0,144.0,155.0,381,5137,0.005889
1,9606.ENSP00000000233,9606.ENSP00000314067,0.0,0.0,0.0,0.0,180.0,0.0,61.0,197.0,381,5062,0.055359
2,9606.ENSP00000000233,9606.ENSP00000263116,0.0,0.0,0.0,62.0,152.0,0.0,101.0,222.0,381,9609,0.084806
3,9606.ENSP00000000233,9606.ENSP00000361263,0.0,0.0,0.0,0.0,161.0,0.0,64.0,181.0,381,2889,0.036514
5,9606.ENSP00000000233,9606.ENSP00000324287,0.0,0.0,0.0,49.0,147.0,0.0,736.0,767.0,381,23527,0.726737


In [ ]:
STRING_protLinks_detailedChannel_complementSubset.to_csv(workDir + 'proteinComplexes/STRING_complementSubset_geneID_conversions.tsv', sep='\t', index=False, header=False)

##generate STRING pairsInfo and predictions files

In [ ]:
#define key variables, e.g. experiment name, filename components, etc.,
MAX_K = 45
targetPercentile = 50.0

In [ ]:
quantile = 0.95

In [ ]:
STRING_protLinks_detailedChannel_complementSubset_above700 = STRING_protLinks_detailedChannel_complementSubset.loc[STRING_protLinks_detailedChannel_complementSubset.combined_score > 700]

In [ ]:
STRING_protLinks_detailedChannel_complementSubset_above700.shape

(178203, 13)

In [ ]:
STRING_protLinks_detailedChannel_complementSubset_above700.head()

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score,protein1_geneID_alias,protein2_geneID_alias,combined_score_scaled
5,9606.ENSP00000000233,9606.ENSP00000324287,0.0,0.0,0.0,49.0,147.0,0.0,736.0,767.0,381,23527,0.726737
96,9606.ENSP00000000233,9606.ENSP00000387286,0.0,0.0,0.0,79.0,248.0,600.0,141.0,730.0,381,5861,0.683157
132,9606.ENSP00000000233,9606.ENSP00000262812,0.0,0.0,0.0,190.0,163.0,600.0,173.0,745.0,381,11316,0.700824
144,9606.ENSP00000000233,9606.ENSP00000158762,0.0,0.0,0.0,0.0,147.0,0.0,736.0,765.0,381,9744,0.724382
145,9606.ENSP00000000233,9606.ENSP00000449270,0.0,0.0,0.0,162.0,210.0,600.0,194.0,757.0,381,22818,0.714959


In [ ]:
cutoff = STRING_protLinks_detailedChannel_complementSubset_above700.combined_score_scaled.quantile(quantile)
print(cutoff)

0.9988221436984688


In [ ]:
numSTRING_top95_pairs = len(STRING_protLinks_detailedChannel_complementSubset_above700.loc[STRING_protLinks_detailedChannel_complementSubset_above700['combined_score_scaled'] >= quantile])
print(numSTRING_top95_pairs/len(STRING_protLinks_detailedChannel_complementSubset_above700))

0.20445783741014462


In [ ]:
pairsInfo_STRING = STRING_protLinks_detailedChannel_complementSubset_above700.loc[STRING_protLinks_detailedChannel_complementSubset_above700['combined_score_scaled'] >= quantile, ['protein1_geneID_alias', 'protein2_geneID_alias']]

In [ ]:
pairsInfo_STRING['label'] = 'NaN'

In [ ]:
pairsInfo_STRING = pairsInfo_STRING[['label', 'protein1_geneID_alias', 'protein2_geneID_alias']]

In [ ]:
pairsInfo_STRING.head()

,label,IDi,IDii
187,NaN,381,375
1082,NaN,381,9727
1490,NaN,4074,3482
1750,NaN,4074,3481
1801,NaN,4074,23163


In [ ]:
pairsInfo_STRING.rename(columns={'protein1_geneID_alias': 'IDi', 'protein2_geneID_alias': 'IDii'}, inplace=True)

In [ ]:
pairsInfo_STRING.shape

(36435, 3)

In [ ]:
pairsInfo_STRING.head()

,label,IDi,IDii
187,NaN,381,375
1082,NaN,381,9727
1490,NaN,4074,3482
1750,NaN,4074,3481
1801,NaN,4074,23163


In [ ]:
pairsInfo_STRING.to_csv(workDir + '/proteinComplexes/STRING_aboveCombined-Score_700_top95.tsv', sep='\t', index=False, header=None)

In [ ]:
pairsPredictions_STRING = STRING_protLinks_detailedChannel_complementSubset_above700.loc[STRING_protLinks_detailedChannel_complementSubset_above700['combined_score_scaled'] >= quantile, 'combined_score_scaled']

In [ ]:
pairsPredictions_STRING.shape

(36435,)

In [ ]:
pairsPredictions_STRING.to_csv(workDir + '/proteinComplexes/STRING_aboveCombined-Score_700_top95_predictions.txt', sep='\t', index=False, header=None)

##run complex building code

In [ ]:
pairsInfo_STRING_path = workDir + '/proteinComplexes/STRING_aboveCombined-Score_700_top95.tsv'
pairsPredictions_STRING_path = workDir + '/proteinComplexes/STRING_aboveCombined_Score_700_top95_predictions.txt'
expName = 'STRING_aboveCombined-Score_700_top95'
pairsInfo_fileSuffix = '.tsv'
pairsScores_fileSuffix = '_predictions'

In [ ]:
cutoff

0.9988221436984688

In [ ]:
pairsInfoFilename = complexesPath + expName + pairsInfo_fileSuffix
pairsPredictionsFilename = complexesPath + expName + pairsScores_fileSuffix

TOP_CUTOFF = cutoff   #0.7740 for 0.05% high-scoring pairs; 0.8663 for 0.02% high-scoring pairs
                      #0.9251 for 0.01% high-scoring pairs; 0.9615 for 0.005% high-scoring pairs
                      #0.9251 for 0.02% high-scoring pairs; 0. for 0.005% high-scoring pairs
                      #0.9840 for 0.002% high-scoring pairs; 0.9898 for 0.001% high-scoring pairs

#Open input file
try:
    infilePairsInfo = open(pairsInfoFilename, "r")
except IOError as e:
    print("<<ERROR>> Unable to open the file", pairsInfoFilename, "\nThis program will be quiting now."), e
    sys.exit()

#Open input file
predictionsFilename = pairsPredictionsFilename + '.txt'
try:
    infileScores = open(predictionsFilename, "r")
except IOError as e:
    print("<<ERROR>> Unable to open the file", predictionsFilename, "\nThis program will be quiting now."), e
    sys.exit()

proteins = {}
restrictedProteins = {}
currComplexes = {}
scores = {}
comparisonMatrix = {}
outfilename = pairsPredictionsFilename + '_candidates_complexesSize_2.txt'
outfile = open(outfilename, 'w')
for line in infileScores:
    line = line.strip()
    tokens = line.split('\t')
    pScore = float(tokens[0])

    infoLine = infilePairsInfo.readline().strip()
    tokens = infoLine.split('\t')
    classLabel = tokens[0]
    prot1ID = tokens[1]
    prot2ID = tokens[2]
    pair1 = (prot1ID, prot2ID)
    pair2 = (prot2ID, prot1ID)
    comparisonMatrix[pair1] = pScore

    if not (prot1ID in proteins):
        proteins[prot1ID] = prot1ID
    if not (prot2ID in proteins):
        proteins[prot2ID] = prot2ID

    temp = []
    if pScore >= TOP_CUTOFF:
        if not (prot1ID in restrictedProteins):
            restrictedProteins[prot1ID] = prot1ID
        if not (prot2ID in restrictedProteins):
            restrictedProteins[prot2ID] = prot2ID

        if not (prot1ID in temp):
            temp.append(prot1ID)
        if not (prot2ID in temp):
            temp.append(prot2ID)
        temp.sort()
        complexID = ','.join(temp)
        if not (complexID in currComplexes):
            outline = complexID + '\t' + str(pScore) + '\n'
            outfile.writelines(outline)
            currComplexes[complexID] = (temp, pScore)
            scores[complexID] = pScore

#Close files
infilePairsInfo.close()
infileScores.close()
outfile.close()
outfilename = pairsPredictionsFilename.split('_features_')[0] + '_candidates_complexesSize_2@' + str(cutoff) + '.gv'
outfile = open(outfilename, 'w')

#Prepare headers
outline = 'graph {' + '\n'
outline += '\t node [shape = circle];' + '\n'
outline += '\t edge [style = solid];' + '\n'
outfile.writelines(outline)
nodeID = 0
protID2nodeID = {}
for protID in restrictedProteins.keys():
    if not (protID in protID2nodeID):
        protID2nodeID[protID] = nodeID
        outline = '\t n' + str(nodeID) + ' [label = p' + str(nodeID) + '];' + '\n'
        outfile.writelines(outline)
        nodeID += 1

outfilenameGraph = pairsPredictionsFilename.split('_features_')[0] + '_candidates_complexesSize_2@' + str(cutoff) + '.graph'
outfileGraph = open(outfilenameGraph, 'w')
outlineGraph = 'p edge ' + str(len(restrictedProteins)) + '\n'
outfileGraph.writelines(outlineGraph)
for complexID, attributes in currComplexes.items():
    prot1ID, prot2ID = complexID.split(',')
    edgeScore = copy.copy(attributes[1])
    if prot1ID in protID2nodeID:
        node1ID = protID2nodeID[prot1ID]
    if prot2ID in protID2nodeID:
        node2ID = protID2nodeID[prot2ID]
    outline = '\t n' + str(node1ID) + ' -- ' + 'n' + str(node2ID) + ' [label = ' + str(edgeScore) + '];' + '\n'
    outfile.writelines(outline)
    outlineGraph = 'e ' + str(prot1ID) + ' ' + str(prot2ID) + ' ' + str(edgeScore) + '\n'
    outfileGraph.writelines(outlineGraph)

    #Prepare footers
    outline = '}' + '\n'
outfile.writelines(outline)

#Close file
outfile.close()
outfileGraph.close()

#return

percentile = np.percentile(list(scores.values()), targetPercentile)
averageWeight = np.mean(list(scores.values()))
stdWeight = np.std(list(scores.values()))
##weightThreshold = averageWeight + (1.0 * stdWeight)
##weightThreshold = averageWeight + (2.0 * stdWeight)
weightThreshold = percentile

print('# of proteins', len(proteins), len(restrictedProteins))
print('Candidate pairwise interactions', len(comparisonMatrix))
print('\t High-scoring pairwise interactions', len(currComplexes))
print('\t Average pairwise score', averageWeight)
print('\t Standard deviation of pairwise scores', stdWeight)
outline = '\t ' + str(targetPercentile) + ' percentile score ' + str(weightThreshold)
print(outline)

k = 3
while len(currComplexes) > 0 and k <= MAX_K:
    complexesCount = 0
    candidateComplexes = copy.copy(currComplexes)
    print (k - 1), len(restrictedProteins)
    ##proteinSet = proteins.keys()
    proteinSet = restrictedProteins.keys()
    restrictedProteins = {}
    scores = {}
    currComplexes = {}
    alreadyPrinted = {}
    ##outfilenameCandidates = outputPATH + pairsPredictionsFilename + '_candidates_complexesSize_' + str(k) + '.txt'
    ##outfileCandidates = open(outfilenameCandidates, 'w')
    outfilenamePredicted = pairsPredictionsFilename + '_predicted_complexesSize_' + str(k-1) + '.txt'
    outfilePredicted = open(outfilenamePredicted, "w")
    for candidateComplex, weight in candidateComplexes.values():
        ##prune = False
        augmented = False
        ##temp = []
        ####temp = copy.copy(candidateComplex)
        ####if weight < TOP_CUTOFF:
        ####continue
        for prot2ID in proteinSet:
            prune = False
            temp = []
            for i in range(len(candidateComplex)):
                prot1ID = candidateComplex[i]
                if prot2ID in candidateComplex:
                    break
                pair1 = (prot1ID, prot2ID)
                pair2 = (prot2ID, prot1ID)
                if pair1 in comparisonMatrix:
                    pScore = comparisonMatrix[pair1]
                elif pair2 in comparisonMatrix:
                    pScore = comparisonMatrix[pair2]
                else:
                    break
                if pScore >= TOP_CUTOFF: #PAIRWISE_INTERACTIONS_CUTOFF:
                    if not (prot1ID in temp):
                        temp.append(prot1ID)
                    if not (prot2ID in temp):
                        temp.append(prot2ID)
                else:
                    prune = True
                    break
            if prune == False and len(temp) == k:
                augmented = True
                temp.sort()
                complexID = ','.join(temp)
                if not (complexID in currComplexes):
                    complexMembers = complexID.split(',')
                    scoresPerPair = []
                    for x in range((len(complexMembers) - 1)):
                        for y in range(x + 1, len(complexMembers)):
                            pairF = (complexMembers[x], complexMembers[y])
                            pairB = (complexMembers[y], complexMembers[x])
                            pairScore = 0.0
                            if pairF in comparisonMatrix:
                                pairScore = comparisonMatrix[pairF]
                            elif pairB in comparisonMatrix:
                                pairScore = comparisonMatrix[pairB]
                            scoresPerPair.append(pairScore)
                    newWeight = min(scoresPerPair)
                    ####newWeight = np.mean(scoresPerPair)
                    ####newWeight = np.percentile(scoresPerPair, 50)
                    if newWeight < TOP_CUTOFF:
                        continue
                        ##outline = complexID
                    ##for pairScore in scoresPerPair:
                        ##outline += '\t' + str(pairScore)
                    ##outline += '\n'
                    ##outfileCandidates.writelines(outline)
                    currComplexes[complexID] = (temp, newWeight)
                    for prot in temp:
                        if not (prot in restrictedProteins):
                            restrictedProteins[prot] = prot
                    scores[complexID] = newWeight
        if not augmented:
            candidateComplex.sort()
            key = ','.join(candidateComplex)
            outline = key + '\t' + str(weight) + '\n'
            outfilePredicted.writelines(outline)
            complexesCount += 1

    #Close outfile
    ##outfileCandidates.close()
    outfilePredicted.close()

    outline = '\t Total of predicted ' + str(k-1) + '-complexes'
    print(outline, complexesCount, len(candidateComplexes))

    outline = 'Candidate ' + str(k) + '-complexes'
    if len(scores) < 1:
        print(outline, len(currComplexes))
        continue
    ##percentile = np.percentile(list(scores.values()), targetPercentile)
    averageWeight = np.mean(list(scores.values()))
    stdWeight = np.std(list(scores.values()))
    weightThreshold = averageWeight + (1.0 * stdWeight)
    ##weightThreshold = averageWeight + (2.0 * stdWeight)
    ##weightThreshold = percentile

    print(outline, len(currComplexes), averageWeight, stdWeight, weightThreshold)

    k = k + 1

# of proteins 7616 3315
Candidate pairwise interactions 36433
	 High-scoring pairwise interactions 7373
	 Average pairwise score 0.999758773495821
	 Standard deviation of pairwise scores 0.00047533139147882625
	 50.0 percentile score 1.0
2
	 Total of predicted 2-complexes 2161 7373
Candidate 3-complexes 18783 0.99979469192721 0.00044684673271506986 1.000241538659925
3
	 Total of predicted 3-complexes 555 18783
Candidate 4-complexes 67962 0.9997957356409486 0.0004459485778744001 1.000241684218823
4
	 Total of predicted 4-complexes 208 67962
Candidate 5-complexes 202404 0.9997739247274844 0.0004638686834759012 1.0002377934109603
5
	 Total of predicted 5-complexes 119 202404
Candidate 6-complexes 470034 0.9997420129991547 0.00048714917843871534 1.0002291621775934
6
	 Total of predicted 6-complexes 120 470034
Candidate 7-complexes 855651 0.9997097795198606 0.0005075530457728667 1.0002173325656334
7
	 Total of predicted 7-complexes 76 855651
Candidate 8-complexes 1236291 0.999682650066285 0

In [ ]:
def findComplexes_corumDifferences(CORUM_complexesData_path, complexesSet_filenamePatt, complexLength_max):

  CORUM_complexesData = pd.read_csv(CORUM_complexesData_path, sep='\t')
  print('The shape of the CORUM data is: {0}'.format(CORUM_complexesData.shape))
  print('printing the head of the CORUM data...')
  print(CORUM_complexesData.head())
  proteinsComplex_compositionCORUM = CORUM_complexesData['subunits(Entrez IDs)']

  proteinsComplex_compositionCORUM_dict = {}
  for complexCORUM_idx in np.arange(len(proteinsComplex_compositionCORUM)):
    try:
      proteinsList = proteinsComplex_compositionCORUM.iloc[complexCORUM_idx].split(';')
      print(len(proteinsList))
      print(proteinsList)
      print(type(proteinsList))
      map_object = map(int, proteinsList)
    except:
      proteinsList = proteinsComplex_compositionCORUM.iloc[complexCORUM_idx].split(';')
      proteinsList.remove('None')
      print(len(proteinsList))
      print(proteinsList)
      map_object = map(int, proteinsList)
    proteinsList = list(map_object)
    proteinsComplex_compositionCORUM_dict[complexCORUM_idx] = proteinsList      

  sharedComplexes = np.zeros((complexLength_max, ))
  j=0
  for complexSize in np.arange(complexLength_max):
    complexesSet = pd.read_csv(complexesSet_filenamePatt + '_' + complexSize + '.txt', sep='\t', engine='python')
    #complexesSet = pd.read_csv(complexesSet_filenamePatt + '_' + complexSize + '.txt', sep=r'\,|\t', engine='python')
    print('The shape of the complexes set for complex size = {0} is: {1}'.format(complexSize, complexesSet.shape))
    print('printing the head of the complexes set for complex size = {0}...'.format(complexSize))
    print(complexesSet.head()) 

    for numComplexes in np.arange(len(complexesSet)):
      complexesCmp = complexesSet.iloc[numComplexes, 0].split(',')
      print(len(complexesCmp))
      map_object = map(int, complexesCmp)
      complexesExp = set(list(map_object))
      for complexCORUM_idx in np.arange(len(proteinsComplex_compositionCORUM)):
        if (len(set(proteinsComplex_compositionCORUM[complexCORUM_idx]) - complexesExp)==0) & (len(complexesExp - set(proteinsComplex_compositionCORUM[complexCORUM_idx]))==0):
          sharedComplexes[j] += 1
          break
    j+=1

  return sharedComplexes

In [ ]:
complexesPath

'/content/drive/My Drive/Colab Notebooks/Primary Research/JLMwSCBC_notebook/proteinComplexes/'

In [ ]:
sharedComplexes_STRING = findComplexes_corumDifferences(complexesPath + 'coreComplexes_03-09-2018_Corum-3.0 .txt', complexesPath + 'STRING_aboveCombined-Score_700_top95_predictions_predicted_complexesSize_', 19)

The shape of the CORUM data is: (3512, 20)
printing the head of the CORUM data...
   ComplexID  ...                             subunits(Protein name)
0          1  ...    B-cell lymphoma 6 protein;Histone deacetylase 4
1          2  ...    B-cell lymphoma 6 protein;Histone deacetylase 5
2          3  ...    B-cell lymphoma 6 protein;Histone deacetylase 7
3          4  ...  Histone acetyltransferase p300;CREB-binding pr...
4          9  ...  Aryl hydrocarbon receptor ;Aryl hydrocarbon re...

[5 rows x 20 columns]
2
['604', '9759']
<class 'list'>
2
['604', '10014']
<class 'list'>
2
['604', '51564']
<class 'list'>
4
['2033', '1387', '8850', '8202']
<class 'list'>
2
['11622', '11863']
<class 'list'>
2
['3257', '89781']
<class 'list'>
3
['79803', '84343', '11234']
<class 'list'>
2
['1040', '80198']
<class 'list'>
6
['8841', '6907', '9611', '2874', '7464', '79718']
<class 'list'>
8
['23557', '2647', '388552', '282991', '63915', '84062', '55330', '26258']
<class 'list'>
4
['11632', '15519', 

ValueError: ignored

#count the number of unique proteins for each clique size

In [ ]:
complexesGenerated_statsColumns=['Cell Line', 'Max Clique Size', 'Clique Size', 'Unique protein count']
complexesGenerated_stats = pd.DataFrame(columns=complexesGenerated_statsColumns)

expName = 'integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV'
cellLines = ['h322m', 'mcf7', 'u251']
for cellLine in cellLines:
  dirPath = modelPerformances_path + expName + '/' + cellLine + \
      '_predictedComplexes/*predicted*txt'
  complexesFiles = glob.glob(dirPath)
  print('The # of complexes files generated for cell line {0} is {1}'.format(
      cellLine, len(complexesFiles)))
  print('___________________________________________________________')

  for file in complexesFiles:
    complexes = pd.read_csv(file, sep=',|\t', usecols=list(
        np.arange(
            int(
                file.split('_')[-1].split('.')[0]
                )
            )
        ), header=None)
    tableRow = pd.DataFrame(data=[cellLine, len(complexesFiles), file.split('_')[-1].split('.')[0], len(complexes.reset_index(drop=True).stack().drop_duplicates())]).T
    tableRow.columns = complexesGenerated_statsColumns
    complexesGenerated_stats = complexesGenerated_stats.append(tableRow)

The # of complexes files generated for cell line h322m is 5
___________________________________________________________
The # of complexes files generated for cell line mcf7 is 6
___________________________________________________________
The # of complexes files generated for cell line u251 is 6
___________________________________________________________


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [ ]:
complexesGenerated_stats.to_excel('/content/complexesGenerated_stats.xlsx')

In [ ]:
#print('The clique size and unique protein count for file {0} is {1} and {2}, respectively'.format(file, file.split('_')[-1].split('.')[0], len(complexes.reset_index(drop=True).stack().drop_duplicates())))

#compare Jose complexes to partition-model based complexes

In [ ]:
JLM_predictedComplexes_0_05 = pd.read_excel('/content/Uncharacterized_protein_complexes.xlsx', sheet_name='top0.05%')

In [ ]:
JLM_predictedComplexes_0_05.head()

,Predicted Complex (geneNames),Predicted Complex (geneIDs),Complex size,Ontology IDs,Ontology Names,Complex Score (min),Adjusted P-value,STRING score (experimental ony),STRING score (combined avg),"Type of CORUM match (0 = predicted complex not in CORUM , 1 = predicted complex identical match in CORUM, predicted complex is strict subset of complex in CORUM )",Overlap fraction of best CORUM complex wrt predicted complex members,Overlap fraction of predicted complex wrt best CORUM complex,Complex members of largest CORUM match,Complex size of largest CORUM match,"Type of hu.MAP match (0 = predicted complex not in hu.MAP , 1 = predicted complex identical match in hu.MAP, predicted complex is strict subset of complex in hu.MAP )",Overlap fraction of best hu.MAP complex wrt predicted complex members,Overlap fraction of predicted complex wrt best hu.MAP complex,Complex members of largest hu.MAP match,Complex size of largest hu.MAP match,Predicted complex proteins in RAS data,Fraction of complex proteins in RAS data,Minimum Pearson Correlation Coeffient between expression of wild-type,Avg WT expression for gene1 of minimum scoring pair,Avg WT expression for gene2 of minimum scoring pair
0,ACTZ;DCTN4;ARP10,10121;51164;55860,3,GO:0005869;GO:0019886;GO:0002495;GO:0002504;GO...,dynactin complex;antigen processing and presen...,0.92817,0.0033,0.870000,0.996667,0,0.0,0.0,NaN,0,2,1.0,0.375,10120;10121;10540;10671;1639;51164;55860;84516,8,1,0.333333,NaN,NaN,NaN
1,ACTR1B;DCTN4;ARP10,10120;51164;55860,3,GO:0005869;GO:0019886;GO:0002495;GO:0002504;GO...,dynactin complex;antigen processing and presen...,0.91995,0.0033,0.862000,0.996333,0,0.0,0.0,NaN,0,2,1.0,0.375,10120;10121;10540;10671;1639;51164;55860;84516,8,0,0.000000,NaN,NaN,NaN
2,NUP62;NUP88;NU214,23636;4927;8021,3,GO:0017056;GO:0005643;GO:0071427;GO:0006406;GO...,structural constituent of nuclear pore;nuclear...,0.89579,0.0099,0.866667,0.999000,0,0.0,0.0,NaN,0,2,1.0,0.375,23165;23511;23636;4927;8021;8480;9631;9688,8,0,0.000000,NaN,NaN,NaN
3,TMED2;TMED10;TMED3,10959;10972;23423,3,GO:0030137;GO:0033116;GO:0006890;GO:0048205;GO...,COPI-coated vesicle;endoplasmic reticulum-Golg...,0.87724,0.0079,0.858000,0.994000,0,0.0,0.0,NaN,0,2,1.0,1.000,10959;10972;23423,3,0,0.000000,NaN,NaN,NaN
4,DCTN6;DCTN4;DCTN5,10671;51164;84516,3,GO:0019886;GO:0002495;GO:0002504;GO:0002478;GO...,antigen processing and presentation of exogeno...,0.86958,0.0326,0.924333,0.998667,0,0.0,0.0,NaN,0,2,1.0,0.375,10120;10121;10540;10671;1639;51164;55860;84516,8,0,0.000000,NaN,NaN,NaN


In [ ]:
JLM_predictedComplexes_0_05.columns

Index(['Predicted Complex (geneNames)', 'Predicted Complex (geneIDs)',
       'Complex size', 'Ontology IDs', 'Ontology Names', 'Complex Score (min)',
       'Adjusted P-value', 'STRING score (experimental ony)',
       'STRING score (combined avg)',
       'Type of CORUM match (0 = predicted complex not in CORUM , 1 = predicted complex identical match in CORUM, predicted complex is strict subset of complex in  CORUM )',
       'Overlap fraction of best CORUM complex wrt predicted complex members',
       'Overlap fraction of predicted complex wrt best  CORUM complex',
       'Complex members of largest CORUM match ',
       'Complex size of largest CORUM match ',
       'Type of hu.MAP match (0 = predicted complex not in hu.MAP , 1 = predicted complex identical match in hu.MAP, predicted complex is strict subset of complex in  hu.MAP )',
       'Overlap fraction of best hu.MAP complex wrt predicted complex members',
       'Overlap fraction of predicted complex wrt best  hu.MAP comple

In [ ]:
complexCounter = 0
complexesAll = dict()
for idx in JLM_predictedComplexes_0_05.index:
  complexesAll[complexCounter] = set(list(map(int, JLM_predictedComplexes_0_05.loc[idx, 'Predicted Complex (geneIDs)'].split(';'))))
  complexCounter+=1

In [ ]:
len(complexesAll)

7758

In [ ]:
complexesAll[0]

{10121, 51164, 55860}

In [ ]:
proteinInd_unionSet = set()

In [ ]:
type(complexesAll)

dict

In [ ]:
for complexSet in complexesAll.values():
  proteinInd_unionSet = proteinInd_unionSet.union(complexSet) 

In [ ]:
len(proteinInd_unionSet)

1135